In [ ]:
#Inversed filters

In [ ]:
#!/usr/bin/env python3
import os
import glob
import sys
import time

import cv2
import torch
import numpy as np
from ultralytics import YOLO

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40  # red tint on rail mask

# Color/size filter (RGB targets; converted to BGR internally)
TARGET_COLORS_RGB = [(119,104,67), (81,42,45)]
TOLERANCE         = 20.0      # color distance (BGR) – increase if too strict
MIN_REGION_SIZE   = 30        # connected-component area in px
MIN_REGION_HEIGHT = 150       # connected-component bbox height in px

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model (load, fuse, warmup)
# =======================
model = YOLO(weights)
try:
    model.fuse()
except Exception:
    pass

_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(
    _dummy, task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, classes=[RAIL_ID],
    verbose=False, half=half
)

# =======================
# FAST color+size filter (vectorized + ROI-cropped)
# =======================
def highlight_rails_mask_only_fast(
    img_bgr: np.ndarray,
    rail_mask: np.ndarray,
    target_colors_rgb: list[tuple[int,int,int]],
    tolerance: float = 30.0,
    min_region_size: int = 50,
    min_region_height: int = 150
) -> np.ndarray:
    """
    Returns boolean mask of rail pixels that:
      (1) color-match any target color within tolerance, AND
      (2) lie inside rail_mask, AND
      (3) pass connected-component size/height filters.
    """
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    # Crop to rail ROI
    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    # Vectorized color match
    targets_bgr = np.array([(b,g,r) for r,g,b in target_colors_rgb], np.float32)
    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - targets_bgr[None, None, :, :]
    dist2 = np.sum(diff * diff, axis=-1)
    color_mask_roi = np.any(dist2 <= (tolerance * tolerance), axis=-1)

    combined_roi = mask_roi & color_mask_roi

    # Connected-component filter
    comp_u8 = combined_roi.astype(np.uint8)
    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(comp_u8, 8)

    filtered_roi = np.zeros_like(combined_roi)
    for lbl in range(1, n_labels):
        area   = stats[lbl, cv2.CC_STAT_AREA]
        height = stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and height >= min_region_height:
            filtered_roi[labels == lbl] = True

    filtered_full[y0:y1, x0:x1] = filtered_roi
    return filtered_full

# =======================
# Per-frame pipeline
# =======================
def process_frame(img_bgr: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Returns:
      - out        : original + red tint on rails + neon-green on unfiltered rails
      - rail_mask  : boolean mask of all rails
    """
    res = model.predict(
        img_bgr, task="segment", imgsz=IMG_SIZE, device=device,
        conf=CONF, iou=IOU, classes=[RAIL_ID],
        max_det=20, verbose=False, half=half
    )[0]

    H, W = img_bgr.shape[:2]
    if res.masks is None:
        return img_bgr.copy(), np.zeros((H, W), bool)

    # union of rail segments
    m = res.masks.data
    union = (m.sum(dim=0) > 0).float().cpu().numpy()
    if union.shape != (H, W):
        union = cv2.resize(union, (W, H), interpolation=cv2.INTER_NEAREST)
    rail_mask = union.astype(bool)

    # compute filtered (original neon-green) mask
    filtered_mask = highlight_rails_mask_only_fast(
        img_bgr, rail_mask,
        TARGET_COLORS_RGB, TOLERANCE,
        MIN_REGION_SIZE, MIN_REGION_HEIGHT
    )

    # inverted mask = rails minus the filtered zones
    inverted_mask = rail_mask & ~filtered_mask

    # compose visualization
    out = img_bgr.copy()

    # red tint for rails
    overlay = out.copy()
    overlay[rail_mask] = (0, 0, 255)
    out = cv2.addWeighted(overlay, ALPHA, out, 1 - ALPHA, 0)

    # neon-green on the *inverse* region
    out[inverted_mask] = (0, 255, 0)

    return out, rail_mask

# =======================
# Main: run & step through
# =======================
if __name__ == "__main__":
    image_dir = f"{home}/SubwaySurfers/train_screenshots"
    paths = sorted(glob.glob(os.path.join(image_dir, "frame_*.jpg")))
    if not paths:
        print("No frames found in", image_dir)
        sys.exit(1)

    for p in paths:
        img = cv2.imread(p)
        if img is None:
            continue

        vis, _ = process_frame(img)

        # show original | inverted neon-green
        combo = np.hstack((img, vis))
        cv2.imshow("Original (Left) | Inverted Rails (Right)", combo)

        # SPACE → next, 'q' → quit
        while True:
            key = cv2.waitKey(0) & 0xFF
            if key == 32:
                break
            elif key == ord('q'):
                cv2.destroyAllWindows()
                sys.exit(0)

        cv2.destroyAllWindows()


In [ ]:
#!/usr/bin/env python3
import os, glob, sys, time
import cv2, torch, numpy as np
from ultralytics import YOLO

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40  # red tint on rail mask

# Color/size filter (RGB targets; converted to BGR internally)
TARGET_COLORS_RGB = [(119,104,67), (81,42,45)]
TOLERANCE         = 20.0      # color distance (BGR) – increase if too strict
MIN_REGION_SIZE   = 30        # connected-component area in px
MIN_REGION_HEIGHT = 150       # connected-component bbox height in px

# Centerline settings
N_LINES           = 3
MIN_GAP_FRAC      = 0.15      # min horizontal separation between lines (fraction of width)
BAND_FRAC         = 0.04      # half-width (as fraction of image width) for local centroid band
Y_TOP_FRAC        = 0.25      # start tracing from this fraction downwards (perspective)
BOTTOM_FOCUS_FRAC = 0.60      # use bottom 60% to score column density
THICKNESS         = 6
PURPLE            = (255, 0, 255)  # BGR

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model (load, fuse, warmup)
# =======================
model = YOLO(weights)
try:
    model.fuse()
except Exception:
    pass

_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(
    _dummy, task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, classes=[RAIL_ID],
    verbose=False, half=half
)

# =======================
# FAST color+size filter (vectorized + ROI-cropped)
# =======================
def highlight_rails_mask_only_fast(
    img_bgr: np.ndarray,
    rail_mask: np.ndarray,
    target_colors_rgb: list[tuple[int,int,int]],
    tolerance: float = 30.0,
    min_region_size: int = 50,
    min_region_height: int = 150
) -> np.ndarray:
    """
    Returns boolean mask of rail pixels that:
      (1) color-match any target color within tolerance, AND
      (2) lie inside rail_mask, AND
      (3) pass connected-component size/height filters.
    """
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    # RGB -> BGR
    targets_bgr = np.array([(b,g,r) for r,g,b in target_colors_rgb], np.float32)
    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - targets_bgr[None, None, :, :]
    dist2 = np.sum(diff * diff, axis=-1)
    color_mask_roi = np.any(dist2 <= (tolerance * tolerance), axis=-1)

    combined_roi = mask_roi & color_mask_roi

    comp_u8 = combined_roi.astype(np.uint8)
    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(comp_u8, 8)

    filtered_roi = np.zeros_like(combined_roi)
    for lbl in range(1, n_labels):
        area   = stats[lbl, cv2.CC_STAT_AREA]
        height = stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and height >= min_region_height:
            filtered_roi[labels == lbl] = True

    filtered_full[y0:y1, x0:x1] = filtered_roi
    return filtered_full

# =======================
# Column-peak picking (no SciPy)
# =======================
def pick_top_columns(mask: np.ndarray, n: int, min_gap: int) -> list[int]:
    """
    Greedily pick up to n columns with highest vertical density, enforcing min_gap separation.
    Uses bottom portion of the image to score columns (more reliable).
    """
    H, W = mask.shape
    y_start = int(H * (1 - BOTTOM_FOCUS_FRAC))
    dens = mask[y_start:, :].sum(axis=0).astype(np.float32)

    order = np.argsort(-dens)  # descending
    chosen = []
    for x in order:
        if dens[x] <= 0:
            break
        if all(abs(int(x) - int(c)) >= min_gap for c in chosen):
            chosen.append(int(x))
            if len(chosen) == n:
                break
    chosen.sort()
    return chosen

def trace_centerline(mask: np.ndarray, x0: int, band_w: int,
                     y_top: int, step: int = 4, smooth: float = 0.6) -> np.ndarray:
    """
    Follow dense green region from bottom to y_top by taking the centroid within [x0-band_w, x0+band_w].
    Returns Nx1x2 array of int points for cv2.polylines.
    """
    H, W = mask.shape
    pts = []
    x_prev = x0

    for y in range(H-1, y_top-1, -step):
        xl = max(0, x_prev - band_w)
        xr = min(W, x_prev + band_w + 1)
        row = mask[y, xl:xr]
        if row.any():
            xs = np.nonzero(row)[0] + xl
            x_c = xs.mean()
            x_prev = int(round(smooth * x_prev + (1.0 - smooth) * x_c))
        # if no pixels, keep previous x_prev (continues straight)
        pts.append([x_prev, y])

    if len(pts) >= 2:
        return np.array(pts, dtype=np.int32).reshape(-1, 1, 2)
    return np.empty((0, 1, 2), dtype=np.int32)

def draw_three_centerlines(out_img: np.ndarray, green_mask: np.ndarray) -> None:
    """
    Find 3 dense green centerlines and draw them in purple on out_img in-place.
    """
    H, W = green_mask.shape
    min_gap = max(12, int(W * MIN_GAP_FRAC))
    band_w  = max(6,  int(W * BAND_FRAC))
    y_top   = int(H * Y_TOP_FRAC)

    xs = pick_top_columns(green_mask, N_LINES, min_gap)
    for x0 in xs:
        poly = trace_centerline(green_mask, x0, band_w, y_top)
        if poly.size > 0:
            cv2.polylines(out_img, [poly], isClosed=False, color=PURPLE, thickness=THICKNESS, lineType=cv2.LINE_AA)

# =======================
# Per-frame pipeline
# =======================
def process_frame(img_bgr: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Returns:
      - out        : original + rails red tint + inverted green + 3 purple centerlines
      - rail_mask  : boolean mask of all rails
    """
    res = model.predict(
        img_bgr, task="segment", imgsz=IMG_SIZE, device=device,
        conf=CONF, iou=IOU, classes=[RAIL_ID],
        max_det=20, verbose=False, half=half
    )[0]

    H, W = img_bgr.shape[:2]
    if res.masks is None:
        return img_bgr.copy(), np.zeros((H, W), bool)

    # union of rail segments
    m = res.masks.data
    union = (m.sum(dim=0) > 0).float().cpu().numpy()
    if union.shape != (H, W):
        union = cv2.resize(union, (W, H), interpolation=cv2.INTER_NEAREST)
    rail_mask = union.astype(bool)

    # compute filtered (original neon-green) mask
    filtered_mask = highlight_rails_mask_only_fast(
        img_bgr, rail_mask,
        TARGET_COLORS_RGB, TOLERANCE,
        MIN_REGION_SIZE, MIN_REGION_HEIGHT
    )

    # inverted mask = rails minus the filtered zones (this is your green background)
    inverted_mask = rail_mask & ~filtered_mask

    # compose visualization
    out = img_bgr.copy()

    # red tint for rails
    overlay = out.copy()
    overlay[rail_mask] = (0, 0, 255)
    out = cv2.addWeighted(overlay, ALPHA, out, 1 - ALPHA, 0)

    # neon-green on the *inverse* region
    out[inverted_mask] = (0, 255, 0)

    # --- draw 3 purple lines in the densest green regions (no overlap) ---
    draw_three_centerlines(out, inverted_mask)

    return out, rail_mask

# =======================
# Main: run & step through
# =======================
if __name__ == "__main__":
    image_dir = f"{home}/SubwaySurfers/train_screenshots"
    paths = sorted(glob.glob(os.path.join(image_dir, "frame_*.jpg")))
    if not paths:
        print("No frames found in", image_dir)
        sys.exit(1)

    for p in paths:
        img = cv2.imread(p)
        if img is None:
            continue

        vis, _ = process_frame(img)

        combo = np.hstack((img, vis))
        cv2.imshow("Original (Left) | Inverted Rails + 3 Purple Lines (Right)", combo)

        # SPACE → next, 'q' → quit
        while True:
            key = cv2.waitKey(0) & 0xFF
            if key == 32:
                break
            elif key == ord('q'):
                cv2.destroyAllWindows()
                sys.exit(0)

        cv2.destroyAllWindows()


In [ ]:
#Heatmap straight 

In [ ]:
#!/usr/bin/env python3
import os, glob, sys, time
import cv2, torch, numpy as np
from ultralytics import YOLO

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40  # red tint on rail mask

# Color/size filter (RGB targets; converted to BGR internally)
TARGET_COLORS_RGB = [(119,104,67), (81,42,45)]
TOLERANCE         = 20.0      # color distance (BGR) – increase if too strict
MIN_REGION_SIZE   = 30        # connected-component area in px
MIN_REGION_HEIGHT = 150       # connected-component bbox height in px

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model (load, fuse, warmup)
# =======================
model = YOLO(weights)
try:
    model.fuse()
except Exception:
    pass

_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(
    _dummy, task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, classes=[RAIL_ID],
    verbose=False, half=half
)

# =======================
# FAST color+size filter (vectorized + ROI-cropped)
# =======================
def highlight_rails_mask_only_fast(
    img_bgr: np.ndarray,
    rail_mask: np.ndarray,
    target_colors_rgb: list[tuple[int,int,int]],
    tolerance: float = 30.0,
    min_region_size: int = 50,
    min_region_height: int = 150
) -> np.ndarray:
    """
    Returns boolean mask of rail pixels that:
      (1) color-match any target color within tolerance, AND
      (2) lie inside rail_mask, AND
      (3) pass connected-component size/height filters.
    """
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    targets_bgr = np.array([(rbg[2], rbg[1], rbg[0]) for rbg in target_colors_rgb],
                           dtype=np.float32)  # (K,3)

    img_f = img_roi.astype(np.float32)                                  # (h,w,3)
    diff  = img_f[:, :, None, :] - targets_bgr[None, None, :, :]        # (h,w,K,3)
    dist2 = np.sum(diff * diff, axis=-1)                                 # (h,w,K)
    color_mask_roi = np.any(dist2 <= (tolerance * tolerance), axis=-1)   # (h,w)

    combined_roi = mask_roi & color_mask_roi

    comp_u8 = combined_roi.astype(np.uint8)
    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(comp_u8, 8)

    filtered_roi = np.zeros_like(combined_roi)
    for lbl in range(1, n_labels):
        area   = stats[lbl, cv2.CC_STAT_AREA]
        height = stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and height >= min_region_height:
            filtered_roi[labels == lbl] = True

    filtered_full[y0:y1, x0:x1] = filtered_roi
    return filtered_full

# =======================
# Heatmap (red-dominance vs green)
# =======================
def red_vs_green_heatmap(red_mask: np.ndarray, green_mask: np.ndarray,
                         blur_ksize: int = 51) -> np.ndarray:
    """
    Build a heatmap where high values (red) indicate regions with more *red* pixels than green,
    and low values (blue) indicate fewer red pixels. Uses a large blur for local density.
    """
    H, W = red_mask.shape
    # convert to float densities then smooth
    k = (blur_ksize, blur_ksize)
    red_d   = cv2.blur(red_mask.astype(np.float32),   k)
    green_d = cv2.blur(green_mask.astype(np.float32), k)
    diff = red_d - green_d  # >0 => red-dominant, <0 => green-dominant

    # normalize to 0..255 with zero centered around ~128 using symmetric scaling
    amax = float(np.max(np.abs(diff))) + 1e-6
    norm = (diff / (2.0 * amax) + 0.5)  # 0..1 with 0.5 at tie
    norm_u8 = np.clip(norm * 255.0, 0, 255).astype(np.uint8)

    heat = cv2.applyColorMap(norm_u8, cv2.COLORMAP_JET)  # blue -> red
    return heat

# =======================
# Per-frame pipeline
# =======================
def process_frame(img_bgr: np.ndarray) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Returns:
      - out_bgr     : original with red rail mask + neon green filtered regions
      - rail_mask   : boolean rail mask
      - heatmap_bgr : red-vs-green heatmap (blue=less red, red=more red)
    """
    res = model.predict(
        img_bgr, task="segment", imgsz=IMG_SIZE, device=device,
        conf=CONF, iou=IOU, classes=[RAIL_ID],
        max_det=20, verbose=False, half=half
    )[0]

    H, W = img_bgr.shape[:2]
    if res.masks is None:
        empty = np.zeros((H, W), bool)
        heat  = red_vs_green_heatmap(empty, empty)
        return img_bgr, empty, heat

    # Union of rail masks
    m = res.masks.data
    union = (m.sum(dim=0) > 0).float().cpu().numpy()
    if union.shape != (H, W):
        union = cv2.resize(union, (W, H), interpolation=cv2.INTER_NEAREST)
    rail_mask = union.astype(bool)

    # Apply fast color/size filter inside rails (these are the green regions)
    filtered_mask = highlight_rails_mask_only_fast(
        img_bgr, rail_mask,
        target_colors_rgb=TARGET_COLORS_RGB,
        tolerance=TOLERANCE,
        min_region_size=MIN_REGION_SIZE,
        min_region_height=MIN_REGION_HEIGHT
    )

    # red area = rails but not filtered
    red_area   = rail_mask & (~filtered_mask)
    green_area = filtered_mask

    # Compose visualization
    out = img_bgr.copy()
    overlay = out.copy()
    overlay[rail_mask] = (0, 0, 255)       # red tint for all rails
    out = cv2.addWeighted(overlay, ALPHA, out, 1 - ALPHA, 0)
    out[filtered_mask] = (0, 255, 0)       # neon green for filtered regions

    # Heatmap below: red-dominance vs green
    heat = red_vs_green_heatmap(red_area, green_area)

    return out, rail_mask, heat

# =======================
# Run over a folder: show side-by-side with heatmap underneath
# =======================
if __name__ == "__main__":
    image_dir = f"{home}/SubwaySurfers/train_screenshots"
    paths = sorted(glob.glob(os.path.join(image_dir, "frame_*.jpg")))
    if not paths:
        print("No frames found in", image_dir)
        sys.exit(1)

    for p in paths:
        img = cv2.imread(p)
        if img is None:
            continue

        vis, _rail, heat = process_frame(img)

        # Top row: original | rails+filtered
        top = np.hstack((img, vis))
        # Bottom: heatmap stretched to same width
        heat_resized = cv2.resize(heat, (top.shape[1], img.shape[0]), interpolation=cv2.INTER_LINEAR)

        display_img = np.vstack((top, heat_resized))
        cv2.imshow("Top: Original (Left) | Rails+Filtered (Right)  |  Bottom: Red-vs-Green Heatmap", display_img)

        # Wait for SPACE (next) or 'q' (quit)
        while True:
            key = cv2.waitKey(0) & 0xFF
            if key == 32:        # SPACE
                break
            elif key == ord('q'):
                cv2.destroyAllWindows()
                sys.exit(0)

        cv2.destroyAllWindows()


In [ ]:
#Heatmap overlay with rail ends

In [ ]:
#!/usr/bin/env python3
import os, glob, sys, time
import cv2, torch, numpy as np
from ultralytics import YOLO

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40  # red tint on rail mask

# Color/size filter (RGB targets; converted to BGR internally)
TARGET_COLORS_RGB = [(119,104,67), (81,42,45)]
TOLERANCE         = 20.0      # color distance (BGR) – increase if too strict
MIN_REGION_SIZE   = 30        # connected-component area in px
MIN_REGION_HEIGHT = 150       # connected-component bbox height in px

# ---- Heatmap→triangle picking params ----
HEAT_BLUR_KSIZE     = 51            # local averaging
RED_SCORE_THRESH    = 220           # 0..255; "dark red" threshold
EXCLUDE_TOP_PX      = 400            # ignore top Y pixels
EXCLUDE_BOTTOM_PX   = 120           # ignore bottom X pixels (from bottom up)
MIN_DARK_RED_AREA   = 1200          # min area (px) for a dark-red blob
TRI_SIZE_PX         = 18            # triangle size
PURPLE              = (255, 0, 255) # BGR

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model (load, fuse, warmup)
# =======================
model = YOLO(weights)
try:
    model.fuse()
except Exception:
    pass

_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(
    _dummy, task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, classes=[RAIL_ID],
    verbose=False, half=half
)

# =======================
# FAST color+size filter (vectorized + ROI-cropped)
# =======================
def highlight_rails_mask_only_fast(
    img_bgr: np.ndarray,
    rail_mask: np.ndarray,
    target_colors_rgb: list[tuple[int,int,int]],
    tolerance: float = 30.0,
    min_region_size: int = 50,
    min_region_height: int = 150
) -> np.ndarray:
    """
    Returns boolean mask of rail pixels that:
      (1) color-match any target color within tolerance, AND
      (2) lie inside rail_mask, AND
      (3) pass connected-component size/height filters.
    """
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    # RGB -> BGR targets
    targets_bgr = np.array([(rbg[2], rbg[1], rbg[0]) for rbg in target_colors_rgb],
                           dtype=np.float32)

    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - targets_bgr[None, None, :, :]
    dist2 = np.sum(diff * diff, axis=-1)
    color_mask_roi = np.any(dist2 <= (tolerance * tolerance), axis=-1)

    combined_roi = mask_roi & color_mask_roi

    comp_u8 = combined_roi.astype(np.uint8)
    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(comp_u8, 8)

    filtered_roi = np.zeros_like(combined_roi)
    for lbl in range(1, n_labels):
        area   = stats[lbl, cv2.CC_STAT_AREA]
        height = stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and height >= min_region_height:
            filtered_roi[labels == lbl] = True

    filtered_full[y0:y1, x0:x1] = filtered_roi
    return filtered_full

# =======================
# Heatmap (red-dominance vs green) + score map
# =======================
def red_vs_green_heatmap(red_mask: np.ndarray, green_mask: np.ndarray,
                         blur_ksize: int = 51) -> tuple[np.ndarray, np.ndarray]:
    """
    Build a heatmap where high values (red) indicate regions with more *red* pixels than green,
    and low values (blue) indicate fewer red pixels. Also returns the normalized score (0..255).
    """
    # convert to float densities then smooth
    k = (blur_ksize, blur_ksize)
    red_d   = cv2.blur(red_mask.astype(np.float32),   k)
    green_d = cv2.blur(green_mask.astype(np.float32), k)
    diff = red_d - green_d  # >0 => red-dominant, <0 => green-dominant

    # normalize to 0..255 with zero centered around ~128 using symmetric scaling
    amax = float(np.max(np.abs(diff))) + 1e-6
    norm = (diff / (2.0 * amax) + 0.5)  # 0..1 with 0.5 at tie
    norm_u8 = np.clip(norm * 255.0, 0, 255).astype(np.uint8)

    heat = cv2.applyColorMap(norm_u8, cv2.COLORMAP_JET)  # blue -> red
    return heat, norm_u8

# =======================
# Draw a small triangle marker with apex at (x, y)
# =======================
def draw_purple_triangle(img: np.ndarray, x: int, y: int, size: int = TRI_SIZE_PX):
    h = int(size * 1.2)
    pts = np.array([
        [x, y],                    # apex
        [x - size, y + h],         # base left
        [x + size, y + h],         # base right
    ], dtype=np.int32)
    cv2.fillConvexPoly(img, pts, PURPLE)
    cv2.polylines(img, [pts.reshape(-1,1,2)], True, (0,0,0), 1, cv2.LINE_AA)

# =======================
# Per-frame pipeline
# =======================
def process_frame(img_bgr: np.ndarray) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Returns:
      - out_bgr     : original with rail tint + green regions + purple triangles at rail ends
      - rail_mask   : boolean rail mask
      - heatmap_bgr : red-vs-green heatmap (blue=more green, red=more red)
    """
    res = model.predict(
        img_bgr, task="segment", imgsz=IMG_SIZE, device=device,
        conf=CONF, iou=IOU, classes=[RAIL_ID],
        max_det=20, verbose=False, half=half
    )[0]

    H, W = img_bgr.shape[:2]
    if res.masks is None:
        empty = np.zeros((H, W), bool)
        heat, _ = red_vs_green_heatmap(empty, empty, blur_ksize=HEAT_BLUR_KSIZE)
        return img_bgr.copy(), empty, heat

    # union of rail segments
    m = res.masks.data
    union = (m.sum(dim=0) > 0).float().cpu().numpy()
    if union.shape != (H, W):
        union = cv2.resize(union, (W, H), interpolation=cv2.INTER_NEAREST)
    rail_mask = union.astype(bool)

    # compute filtered (green) mask
    filtered_mask = highlight_rails_mask_only_fast(
        img_bgr, rail_mask,
        TARGET_COLORS_RGB, TOLERANCE,
        MIN_REGION_SIZE, MIN_REGION_HEIGHT
    )

    # red area = rails minus the filtered zones
    red_area   = rail_mask & (~filtered_mask)
    green_area = filtered_mask

    # Compose visualization
    orig_marked = img_bgr.copy()             # triangles go on ORIGINAL
    out = img_bgr.copy()                      # rails viz on the right panel
    overlay = out.copy()
    overlay[rail_mask] = (0, 0, 255)
    out = cv2.addWeighted(overlay, ALPHA, out, 1 - ALPHA, 0)
    out[filtered_mask] = (0, 255, 0)

    # Heatmap + score (0..255, 255 = most red-dominant)
    heat, score = red_vs_green_heatmap(red_area, green_area, blur_ksize=HEAT_BLUR_KSIZE)

    # ---- pick dark-red blobs within Y-range and mark their uppermost point ----
    score_mask = (score >= RED_SCORE_THRESH).astype(np.uint8)

    # limit vertical search band
    top = max(0, EXCLUDE_TOP_PX)
    bot = max(0, EXCLUDE_BOTTOM_PX)
    score_mask[:top, :] = 0
    if bot > 0:
        score_mask[H - bot:, :] = 0

    # remove tiny speckle
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 9))
    score_mask = cv2.morphologyEx(score_mask, cv2.MORPH_OPEN, kernel, iterations=1)

    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(score_mask, 8)
    for lbl in range(1, n_labels):
        area = stats[lbl, cv2.CC_STAT_AREA]
        if area < MIN_DARK_RED_AREA:
            continue

        # uppermost y in this blob
        mask_lbl = (labels == lbl)
        ys, xs = np.where(mask_lbl)
        y_min = int(ys.min())

        # x at the top-most row for stability (mean of that row's xs)
        xs_top = xs[ys == y_min]
        x_mid = int(np.clip(int(np.round(xs_top.mean())), 0, W - 1))

        # draw triangle on the ORIGINAL image
        draw_purple_triangle(orig_marked, x_mid, y_min, TRI_SIZE_PX)

    return (orig_marked, rail_mask, heat), out  # return both left(original+triangles) and right(viz), plus heat

# =======================
# Run over a folder: show side-by-side with heatmap underneath
# =======================
if __name__ == "__main__":
    image_dir = f"{home}/SubwaySurfers/train_screenshots"
    paths = sorted(glob.glob(os.path.join(image_dir, "frame_*.jpg")))
    if not paths:
        print("No frames found in", image_dir)
        sys.exit(1)

    for p in paths:
        img = cv2.imread(p)
        if img is None:
            continue

        (orig_with_tri, _rail, heat), rails_viz = process_frame(img)

        # Top row: ORIGINAL(with triangles) | rails+filtered viz
        top = np.hstack((orig_with_tri, rails_viz))

        # Bottom: heatmap resized to match width
        heat_resized = cv2.resize(heat, (top.shape[1], img.shape[0]), interpolation=cv2.INTER_LINEAR)

        display_img = np.vstack((top, heat_resized))
        cv2.imshow("Top: Original + Rail-End Triangles (Left) | Rails+Filtered (Right)  |  Bottom: Red-vs-Green Heatmap", display_img)

        # SPACE → next, 'q' → quit
        while True:
            key = cv2.waitKey(0) & 0xFF
            if key == 32:
                break
            elif key == ord('q'):
                cv2.destroyAllWindows()
                sys.exit(0)

        cv2.destroyAllWindows()


In [ ]:
#PEAK CODE BELOW

In [ ]:
#!/usr/bin/env python3
import os, glob, sys, time
import cv2, torch, numpy as np
from ultralytics import YOLO

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40  # red tint on rail mask

# Color/size filter (RGB targets; converted to BGR internally)
TARGET_COLORS_RGB = [(119,104,67), (81,42,45)]
TOLERANCE         = 20.0
MIN_REGION_SIZE   = 30
MIN_REGION_HEIGHT = 150

# ---- Heatmap→triangle picking params ----
HEAT_BLUR_KSIZE       = 51            # local averaging window
RED_SCORE_THRESH      = 220           # 0..255; "dark red" threshold
EXCLUDE_TOP_FRAC      = 0.40          # exclude top 10% of the image (before analysis)
EXCLUDE_BOTTOM_FRAC   = 0.15          # exclude bottom 15% of the image (before analysis)
MIN_DARK_RED_AREA     = 1200          # min area (px) for a dark-red blob
TRI_SIZE_PX           = 18            # triangle size
PURPLE                = (255, 0, 255) # BGR

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model (load, fuse, warmup)
# =======================
model = YOLO(weights)
try:
    model.fuse()
except Exception:
    pass

_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(
    _dummy, task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, classes=[RAIL_ID],
    verbose=False, half=half
)

# =======================
# FAST color+size filter (vectorized + ROI-cropped)
# =======================
def highlight_rails_mask_only_fast(
    img_bgr: np.ndarray,
    rail_mask: np.ndarray,
    target_colors_rgb: list[tuple[int,int,int]],
    tolerance: float = 30.0,
    min_region_size: int = 50,
    min_region_height: int = 150
) -> np.ndarray:
    """
    Boolean mask of rail pixels that (1) color-match within tolerance,
    (2) lie inside rail_mask, and (3) pass connected-component size/height filters.
    """
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    targets_bgr = np.array([(rbg[2], rbg[1], rbg[0]) for rbg in target_colors_rgb],
                           dtype=np.float32)

    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - targets_bgr[None, None, :, :]
    dist2 = np.sum(diff * diff, axis=-1)
    color_mask_roi = np.any(dist2 <= (tolerance * tolerance), axis=-1)

    combined_roi = mask_roi & color_mask_roi

    comp_u8 = combined_roi.astype(np.uint8)
    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(comp_u8, 8)

    filtered_roi = np.zeros_like(combined_roi)
    for lbl in range(1, n_labels):
        area   = stats[lbl, cv2.CC_STAT_AREA]
        height = stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and height >= min_region_height:
            filtered_roi[labels == lbl] = True

    filtered_full[y0:y1, x0:x1] = filtered_roi
    return filtered_full

# =======================
# Heatmap (red-dominance vs green) + score map
# =======================
def red_vs_green_heatmap(red_mask: np.ndarray, green_mask: np.ndarray,
                         blur_ksize: int = 51) -> tuple[np.ndarray, np.ndarray]:
    """
    Heatmap (JET) and normalized score 0..255 where higher means more *red* than green.
    """
    k = (blur_ksize, blur_ksize)
    red_d   = cv2.blur(red_mask.astype(np.float32),   k)
    green_d = cv2.blur(green_mask.astype(np.float32), k)
    diff = red_d - green_d  # >0 => red-dominant, <0 => green-dominant

    amax = float(np.max(np.abs(diff))) + 1e-6
    norm = (diff / (2.0 * amax) + 0.5)  # 0..1 with 0.5 at tie
    norm_u8 = np.clip(norm * 255.0, 0, 255).astype(np.uint8)

    heat = cv2.applyColorMap(norm_u8, cv2.COLORMAP_JET)
    return heat, norm_u8

# =======================
# Draw a small triangle marker with apex at (x, y)
# =======================
def draw_purple_triangle(img: np.ndarray, x: int, y: int, size: int = TRI_SIZE_PX):
    h = int(size * 1.2)
    pts = np.array([
        [x, y],                    # apex
        [x - size, y + h],         # base left
        [x + size, y + h],         # base right
    ], dtype=np.int32)
    cv2.fillConvexPoly(img, pts, PURPLE)
    cv2.polylines(img, [pts.reshape(-1,1,2)], True, (0,0,0), 1, cv2.LINE_AA)

# =======================
# Per-frame pipeline
# =======================
def process_frame(img_bgr: np.ndarray) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Returns:
      - orig_with_tri : ORIGINAL image with purple triangles at uppermost points of dark-red heat regions
      - rail_mask     : boolean rail mask
      - heatmap_bgr   : red-vs-green heatmap (blue=more green, red=more red)
    """
    res = model.predict(
        img_bgr, task="segment", imgsz=IMG_SIZE, device=device,
        conf=CONF, iou=IOU, classes=[RAIL_ID],
        max_det=20, verbose=False, half=half
    )[0]

    H, W = img_bgr.shape[:2]
    if res.masks is None:
        empty = np.zeros((H, W), bool)
        heat, _ = red_vs_green_heatmap(empty, empty, blur_ksize=HEAT_BLUR_KSIZE)
        return img_bgr.copy(), empty, heat

    # union of rail segments
    m = res.masks.data
    union = (m.sum(dim=0) > 0).float().cpu().numpy()
    if union.shape != (H, W):
        union = cv2.resize(union, (W, H), interpolation=cv2.INTER_NEAREST)
    rail_mask = union.astype(bool)

    # compute filtered (green) mask
    filtered_mask = highlight_rails_mask_only_fast(
        img_bgr, rail_mask,
        TARGET_COLORS_RGB, TOLERANCE,
        MIN_REGION_SIZE, MIN_REGION_HEIGHT
    )

    # red area = rails minus the filtered zones
    red_area   = rail_mask & (~filtered_mask)
    green_area = filtered_mask

    # Heatmap + score (0..255, 255 = most red-dominant)
    heat, score = red_vs_green_heatmap(red_area, green_area, blur_ksize=HEAT_BLUR_KSIZE)

    # ---- percentage-based exclusion applied BEFORE analysis for triangle plotting ----
    top_rows = int(H * max(0.0, min(1.0, EXCLUDE_TOP_FRAC)))
    bot_rows = int(H * max(0.0, min(1.0, EXCLUDE_BOTTOM_FRAC)))
    score[:top_rows, :] = 0
    if bot_rows > 0:
        score[H - bot_rows:, :] = 0

    # threshold dark-red regions
    score_mask = (score >= RED_SCORE_THRESH).astype(np.uint8)

    # denoise small speckle
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 9))
    score_mask = cv2.morphologyEx(score_mask, cv2.MORPH_OPEN, kernel, iterations=1)

    # find blobs and mark the uppermost point per blob
    orig_marked = img_bgr.copy()
    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(score_mask, 8)
    for lbl in range(1, n_labels):
        area = stats[lbl, cv2.CC_STAT_AREA]
        if area < MIN_DARK_RED_AREA:
            continue
        mask_lbl = (labels == lbl)
        ys, xs = np.where(mask_lbl)
        y_min = int(ys.min())
        xs_top = xs[ys == y_min]
        x_mid = int(np.clip(int(np.round(xs_top.mean())), 0, W - 1))
        draw_purple_triangle(orig_marked, x_mid, y_min, TRI_SIZE_PX)

    return orig_marked, rail_mask, heat

# =======================
# Run over a folder: show side-by-side with heatmap underneath
# =======================
if __name__ == "__main__":
    image_dir = f"{home}/SubwaySurfers/train_screenshots"
    paths = sorted(glob.glob(os.path.join(image_dir, "frame_*.jpg")))
    if not paths:
        print("No frames found in", image_dir)
        sys.exit(1)

    for p in paths:
        img = cv2.imread(p)
        if img is None:
            continue

        orig_with_tri, _rail, heat = process_frame(img)

        # For reference, also render rails+filtered (right pane)
        # (Optional: remove this block if you only want original + heatmap)
        overlay = img.copy()
        res_rails = model.predict(
            img, task="segment", imgsz=IMG_SIZE, device=device,
            conf=CONF, iou=IOU, classes=[RAIL_ID],
            max_det=20, verbose=False, half=half
        )[0]
        if res_rails.masks is not None:
            m = res_rails.masks.data
            union = (m.sum(dim=0) > 0).float().cpu().numpy()
            if union.shape != img.shape[:2]:
                union = cv2.resize(union, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)
            rail_mask_viz = union.astype(bool)
            overlay2 = img.copy()
            overlay2[rail_mask_viz] = (0, 0, 255)
            right = cv2.addWeighted(overlay2, ALPHA, img, 1 - ALPHA, 0)
        else:
            right = img.copy()

        top = np.hstack((orig_with_tri, right))
        heat_resized = cv2.resize(heat, (top.shape[1], img.shape[0]), interpolation=cv2.INTER_LINEAR)
        display_img = np.vstack((top, heat_resized))

        cv2.imshow("Top: Original + Purple Rail-End Triangles (Left) | Rails Viz (Right)  |  Bottom: Red-vs-Green Heatmap", display_img)

        # SPACE → next, 'q' → quit
        while True:
            key = cv2.waitKey(0) & 0xFF
            if key == 32:
                break
            elif key == ord('q'):
                cv2.destroyAllWindows()
                sys.exit(0)

        cv2.destroyAllWindows()


In [ ]:
#Speed analysis

In [ ]:
#!/usr/bin/env python3
"""
Benchmark rail-processing pipeline.
Processes an entire folder of frames *without* any GUI,
collects per-frame runtimes, and prints timing statistics.
"""

import os, glob, sys, time
import cv2, torch, numpy as np
from ultralytics import YOLO

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40  # red tint on rail mask

# Color/size filter (RGB targets; converted to BGR internally)
TARGET_COLORS_RGB = [(119,104,67), (81,42,45)]
TOLERANCE         = 20.0
MIN_REGION_SIZE   = 30
MIN_REGION_HEIGHT = 150

# ---- Heatmap→triangle picking params ----
HEAT_BLUR_KSIZE       = 51
RED_SCORE_THRESH      = 220
EXCLUDE_TOP_FRAC      = 0.40   # ignore top 40 % before triangle search
EXCLUDE_BOTTOM_FRAC   = 0.15   # ignore bottom 15 %
MIN_DARK_RED_AREA     = 1200
TRI_SIZE_PX           = 18
PURPLE                = (255, 0, 255)

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model (load, fuse, warm-up)
# =======================
model = YOLO(weights)
try:
    model.fuse()
except Exception:
    pass

_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(
    _dummy, task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, classes=[RAIL_ID],
    verbose=False, half=half
)

# =======================
# Helpers
# =======================
def highlight_rails_mask_only_fast(
    img_bgr: np.ndarray,
    rail_mask: np.ndarray,
    target_colors_rgb: list[tuple[int,int,int]],
    tolerance: float = 30.0,
    min_region_size: int = 50,
    min_region_height: int = 150
) -> np.ndarray:
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    targets_bgr = np.array([(rbg[2], rbg[1], rbg[0]) for rbg in target_colors_rgb],
                           dtype=np.float32)

    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - targets_bgr[None, None, :, :]
    dist2 = np.sum(diff * diff, axis=-1)
    color_mask_roi = np.any(dist2 <= (tolerance * tolerance), axis=-1)

    combined_roi = mask_roi & color_mask_roi

    comp_u8 = combined_roi.astype(np.uint8)
    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(comp_u8, 8)

    filtered_roi = np.zeros_like(combined_roi)
    for lbl in range(1, n_labels):
        area   = stats[lbl, cv2.CC_STAT_AREA]
        height = stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and height >= min_region_height:
            filtered_roi[labels == lbl] = True

    filtered_full[y0:y1, x0:x1] = filtered_roi
    return filtered_full


def red_vs_green_score(red_mask: np.ndarray,
                       green_mask: np.ndarray,
                       blur_ksize: int = 51) -> np.ndarray:
    """Return a 0-255 score map (higher = more red-dominant)."""
    k = (blur_ksize, blur_ksize)
    r = cv2.blur(red_mask.astype(np.float32), k)
    g = cv2.blur(green_mask.astype(np.float32), k)
    diff = r - g
    amax = float(np.max(np.abs(diff))) + 1e-6
    norm = (diff / (2.0 * amax) + 0.5)          # 0..1
    return np.clip(norm * 255.0, 0, 255).astype(np.uint8)


def process_frame(img_bgr: np.ndarray):
    """Process one frame; returns nothing (we only benchmark)."""
    res = model.predict(
        img_bgr, task="segment", imgsz=IMG_SIZE, device=device,
        conf=CONF, iou=IOU, classes=[RAIL_ID],
        max_det=20, verbose=False, half=half
    )[0]

    if res.masks is None:
        return

    H, W = img_bgr.shape[:2]
    union = (res.masks.data.sum(dim=0) > 0).float().cpu().numpy()
    if union.shape != (H, W):
        union = cv2.resize(union, (W, H), interpolation=cv2.INTER_NEAREST)
    rail_mask = union.astype(bool)

    green = highlight_rails_mask_only_fast(
        img_bgr, rail_mask, TARGET_COLORS_RGB,
        TOLERANCE, MIN_REGION_SIZE, MIN_REGION_HEIGHT
    )
    red   = rail_mask & ~green

    score = red_vs_green_score(red, green, HEAT_BLUR_KSIZE)

    # Exclusion before any potential triangle logic (kept for timing parity)
    top_rows = int(H * EXCLUDE_TOP_FRAC)
    bot_rows = int(H * EXCLUDE_BOTTOM_FRAC)
    score[:top_rows, :] = 0
    if bot_rows:
        score[H - bot_rows:, :] = 0

    # Extract blobs, but we do NOT draw anything for the benchmark
    mask = (score >= RED_SCORE_THRESH).astype(np.uint8)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 9))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    cv2.connectedComponentsWithStats(mask, 8)  # just to emulate same workload


# =======================
# Benchmark loop
# =======================
if __name__ == "__main__":
    image_dir = f"{home}/SubwaySurfers/train_screenshots"
    paths = sorted(glob.glob(os.path.join(image_dir, "frame_*.jpg")))
    if not paths:
        print("No frames found in", image_dir)
        sys.exit(1)

    per_frame = []
    t0_all = time.perf_counter()

    for p in paths[::5]:
        img = cv2.imread(p)
        if img is None:
            continue
        t0 = time.perf_counter()
        process_frame(img)
        per_frame.append(time.perf_counter() - t0)

    total = time.perf_counter() - t0_all
    per_frame_np = np.array(per_frame)

    # Print timing statistics
    print(f"Frames processed  : {len(per_frame)}")
    print(f"Total time (s)    : {total:8.3f}")
    print(f"Average/frame (s) : {per_frame_np.mean():8.4f}")
    print(f"Median/frame (s)  : {np.median(per_frame_np):8.4f}")
    print(f"Min/frame (s)     : {per_frame_np.min():8.4f}")
    print(f"Max/frame (s)     : {per_frame_np.max():8.4f}")


In [ ]:
#!/usr/bin/env python3
"""
Visualise *all* segmentation masks.

• Rails are still tinted red (right-hand panel) and used for the
  heat-map + purple-triangle logic exactly as before.
• Every other detected mask is over-laid on the *original* frame
  (left-hand panel) with a distinct solid colour so you can see
  what the network is segmenting besides the rails.
"""

import os, glob, sys, time
import cv2, torch, numpy as np
from ultralytics import YOLO

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9                     # class-id for rails in the custom model

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40                  # red tint on rail mask (right pane)

# Colour/size filter (RGB targets; converted to BGR internally)
TARGET_COLORS_RGB = [(119,104,67), (81,42,45)]
TOLERANCE         = 20.0
MIN_REGION_SIZE   = 30
MIN_REGION_HEIGHT = 150

# ---- Heat-map → triangle params ----
HEAT_BLUR_KSIZE     = 51
RED_SCORE_THRESH    = 220
EXCLUDE_TOP_FRAC    = 0.40   # ignore top   40 % before triangle search
EXCLUDE_BOTTOM_FRAC = 0.15   # ignore bottom 15 %
MIN_DARK_RED_AREA   = 1200
TRI_SIZE_PX         = 18
PURPLE              = (255, 0, 255)      # triangle colour (BGR)

# Palette for **other** masks (cyclic if > len)
OTHER_COLOURS = [
    (  0,128,255),   # orange
    (  0,255,255),   # yellow
    ( 60,180, 75),   # green
    (255,  0,127),   # pink
    (255,255,  0),   # cyan
    (128,  0,255),   # violet
    (255,128,  0),   # sky blue
]

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model (load, fuse, warm-up)
# =======================
model = YOLO(weights)
try:
    model.fuse()
except Exception:
    pass

# one dummy forward so first real frame is fast
_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
model.predict(_dummy, task="segment", imgsz=IMG_SIZE, device=device,
              conf=CONF, iou=IOU, verbose=False, half=half)

# ------------------------------------------------------------------
# Helper functions (unchanged rail filter + heat-map)
# ------------------------------------------------------------------
def highlight_rails_mask_only_fast(img_bgr, rail_mask, target_colors_rgb,
                                   tolerance=30.0,
                                   min_region_size=50,
                                   min_region_height=150):
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    targets_bgr = np.array([(r,g,b)[::-1] for r,g,b in target_colors_rgb],
                           dtype=np.float32)  # RGB->BGR swap
    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - targets_bgr[None, None, :, :]
    dist2 = np.sum(diff*diff, axis=-1)
    colour_hit = np.any(dist2 <= tolerance**2, axis=-1)

    combined = colour_hit & mask_roi
    comp = combined.astype(np.uint8)
    n, lbls, stats, _ = cv2.connectedComponentsWithStats(comp, 8)

    good = np.zeros_like(combined)
    for lbl in range(1, n):
        area, h = stats[lbl, cv2.CC_STAT_AREA], stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and h >= min_region_height:
            good[lbls == lbl] = True

    filtered_full[y0:y1, x0:x1] = good
    return filtered_full


def red_vs_green_score(red_mask, green_mask, blur_ksize=51):
    k = (blur_ksize, blur_ksize)
    r = cv2.blur(red_mask.astype(np.float32), k)
    g = cv2.blur(green_mask.astype(np.float32), k)
    diff = r - g
    amax = float(np.max(np.abs(diff))) + 1e-6
    norm = (diff / (2*amax) + 0.5)
    return np.clip(norm * 255, 0, 255).astype(np.uint8)


def draw_triangle(img, x, y, size=TRI_SIZE_PX, colour=PURPLE):
    h = int(size * 1.2)
    pts = np.array([[x, y],
                    [x-size, y+h],
                    [x+size, y+h]], np.int32)
    cv2.fillConvexPoly(img, pts, colour)
    cv2.polylines(img, [pts.reshape(-1,1,2)], True, (0,0,0), 1, cv2.LINE_AA)

# ------------------------------------------------------------------
# Main per-frame pipeline
# ------------------------------------------------------------------
def process_frame(img_bgr):
    res = model.predict(img_bgr, task="segment", imgsz=IMG_SIZE,
                        device=device, conf=CONF, iou=IOU,
                        max_det=30, verbose=False, half=half)[0]

    H, W = img_bgr.shape[:2]
    if res.masks is None:                         # no detections
        return img_bgr.copy(), img_bgr.copy(), np.zeros_like(img_bgr)

    masks = res.masks.data.cpu().numpy()          # (N, h, w)
    classes = res.masks.cls.cpu().numpy() if hasattr(res.masks, "cls") \
              else res.boxes.cls.cpu().numpy()    # fallback
    h_m, w_m = masks.shape[1:]

    # ---------------- Rails mask / green filter / heat-map ------------
    rail_union = np.zeros((h_m, w_m), bool)
    for m, c in zip(masks, classes):
        if int(c) == RAIL_ID:
            rail_union |= m.astype(bool)

    rail_mask = rail_union
    if rail_mask.shape != (H, W):
        rail_mask = cv2.resize(rail_mask.astype(np.uint8), (W, H),
                               interpolation=cv2.INTER_NEAREST).astype(bool)

    green = highlight_rails_mask_only_fast(img_bgr, rail_mask,
                                           TARGET_COLORS_RGB,
                                           TOLERANCE,
                                           MIN_REGION_SIZE,
                                           MIN_REGION_HEIGHT)
    red = rail_mask & ~green
    score = red_vs_green_score(red, green, HEAT_BLUR_KSIZE)

    # exclusion
    top_ex = int(H * EXCLUDE_TOP_FRAC)
    bot_ex = int(H * EXCLUDE_BOTTOM_FRAC)
    score[:top_ex, :] = 0
    if bot_ex: score[H-bot_ex:, :] = 0

    dark = (score >= RED_SCORE_THRESH).astype(np.uint8)
    dark = cv2.morphologyEx(dark, cv2.MORPH_OPEN,
                            cv2.getStructuringElement(cv2.MORPH_RECT, (5,9)),
                            iterations=1)

    # ---------------- left panel: original + other-mask colours + triangles ----
    left = img_bgr.copy()

    # Overlay OTHER masks (everything that's NOT RAIL_ID)
    overlay = left.copy()
    colour_idx = 0
    for m, c in zip(masks, classes):
        if int(c) == RAIL_ID:
            continue
        mask_full = m
        if mask_full.shape != (H, W):
            mask_full = cv2.resize(mask_full.astype(np.uint8), (W, H),
                                   interpolation=cv2.INTER_NEAREST).astype(bool)
        colour = OTHER_COLOURS[colour_idx % len(OTHER_COLOURS)]
        overlay[mask_full] = colour
        colour_idx += 1

    # alpha-blend overlay on left frame
    left = cv2.addWeighted(overlay, 0.6, left, 0.4, 0)

    # Purple triangles (after overlay so they stay visible)
    n_lbl, lbl_mat, stats, _ = cv2.connectedComponentsWithStats(dark, 8)
    for lbl in range(1, n_lbl):
        if stats[lbl, cv2.CC_STAT_AREA] < MIN_DARK_RED_AREA:
            continue
        ys, xs = np.where(lbl_mat == lbl)
        y_top = ys.min()
        x_mid = int(xs[ys == y_top].mean())
        draw_triangle(left, x_mid, y_top)

    # ---------------- right panel: rails tinted red -------------------
    right = img_bgr.copy()
    rails_tinted = right.copy()
    if rail_mask.shape != right.shape[:2]:
        rail_mask = rail_mask.astype(np.uint8)
    rails_tinted[rail_mask] = (0,0,255)
    right = cv2.addWeighted(rails_tinted, ALPHA, right, 1-ALPHA, 0)
    right[green] = (0,255,0)

    # ---------------- heat-map visual ----------------
    heat_col = cv2.applyColorMap(score, cv2.COLORMAP_JET)
    heat_col = cv2.resize(heat_col, (left.shape[1]+right.shape[1], H),
                          interpolation=cv2.INTER_LINEAR)

    return left, right, heat_col

# ------------------------------------------------------------------
# Drive over folder
# ------------------------------------------------------------------
if __name__ == "__main__":
    folder = f"{home}/SubwaySurfers/train_screenshots"
    paths = sorted(glob.glob(os.path.join(folder, "frame_*.jpg")))
    if not paths:
        print("No frames found", file=sys.stderr); sys.exit(1)

    for p in paths:
        frame = cv2.imread(p);  t0 = time.time()
        left, right, heat = process_frame(frame)
        top = np.hstack((left, right))
        canvas = np.vstack((top, heat))
        cv2.imshow("Left: Original + coloured other-masks + triangles | "
                   "Right: Rails tinted   |  Bottom: heat-map", canvas)
        print(f"{os.path.basename(p):>20s}   proc {1000*(time.time()-t0):5.1f} ms")
        key = cv2.waitKey(0) & 0xFF
        if key in (ord('q'), 27):
            break
    cv2.destroyAllWindows()


In [ ]:
#!/usr/bin/env python3
"""
Visualise *all* segmentation masks with labelled overlays.

• Rails are still tinted red (right-hand panel) and used for the
  heat-map + purple-triangle logic exactly as before.
• Every other detected mask is over-laid on the *original* frame
  with its true label and a consistent colour.
"""

import os, glob, sys, time
import cv2, torch, numpy as np
from ultralytics import YOLO

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40

TARGET_COLORS_RGB = [(119,104,67), (81,42,45)]
TOLERANCE         = 20.0
MIN_REGION_SIZE   = 30
MIN_REGION_HEIGHT = 150

HEAT_BLUR_KSIZE     = 51
RED_SCORE_THRESH    = 220
EXCLUDE_TOP_FRAC    = 0.40
EXCLUDE_BOTTOM_FRAC = 0.15
MIN_DARK_RED_AREA   = 1200
TRI_SIZE_PX         = 18
PURPLE              = (255, 0, 255)

obstacle_classes = {
    0: "BOOTS", 1: "GREYTRAIN", 2: "HIGHBARRIER1", 3: "JUMP", 4: "LOWBARRIER1",
    5: "LOWBARRIER2", 6: "ORANGETRAIN", 7: "PILLAR", 8: "RAMP", 9: "RAILS",
    10: "SIDEWALK", 11: "YELLOWTRAIN"
}

CLASS_COLOURS = {
    0: (255, 255, 0), 1: (192, 192, 192), 2: (0, 128, 255), 3: (0, 255, 0),
    4: (255, 0, 255), 5: (0, 255, 255), 6: (255, 128, 0), 7: (128, 0, 255),
    8: (0, 0, 128), 10: (128, 128, 0), 11: (255, 255, 102)
}

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

model = YOLO(weights)
try: model.fuse()
except Exception: pass
_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
model.predict(_dummy, task="segment", imgsz=IMG_SIZE, device=device,
              conf=CONF, iou=IOU, verbose=False, half=half)

# ------------------------------------------------------------------
def highlight_rails_mask_only_fast(img_bgr, rail_mask, target_colors_rgb,
                                   tolerance=30.0,
                                   min_region_size=50,
                                   min_region_height=150):
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any(): return filtered_full

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    targets_bgr = np.array([(r,g,b)[::-1] for r,g,b in target_colors_rgb], dtype=np.float32)
    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - targets_bgr[None, None, :, :]
    dist2 = np.sum(diff*diff, axis=-1)
    colour_hit = np.any(dist2 <= tolerance**2, axis=-1)

    combined = colour_hit & mask_roi
    comp = combined.astype(np.uint8)
    n, lbls, stats, _ = cv2.connectedComponentsWithStats(comp, 8)

    good = np.zeros_like(combined)
    for lbl in range(1, n):
        area, h = stats[lbl, cv2.CC_STAT_AREA], stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and h >= min_region_height:
            good[lbls == lbl] = True

    filtered_full[y0:y1, x0:x1] = good
    return filtered_full


def red_vs_green_score(red_mask, green_mask, blur_ksize=51):
    k = (blur_ksize, blur_ksize)
    r = cv2.blur(red_mask.astype(np.float32), k)
    g = cv2.blur(green_mask.astype(np.float32), k)
    diff = r - g
    amax = float(np.max(np.abs(diff))) + 1e-6
    norm = (diff / (2*amax) + 0.5)
    return np.clip(norm * 255, 0, 255).astype(np.uint8)


def draw_triangle(img, x, y, size=TRI_SIZE_PX, colour=PURPLE):
    h = int(size * 1.2)
    pts = np.array([[x, y], [x-size, y+h], [x+size, y+h]], np.int32)
    cv2.fillConvexPoly(img, pts, colour)
    cv2.polylines(img, [pts.reshape(-1,1,2)], True, (0,0,0), 1, cv2.LINE_AA)

# ------------------------------------------------------------------
def process_frame(img_bgr):
    res = model.predict(img_bgr, task="segment", imgsz=IMG_SIZE,
                        device=device, conf=CONF, iou=IOU,
                        max_det=30, verbose=False, half=half)[0]

    H, W = img_bgr.shape[:2]
    if res.masks is None:
        return img_bgr.copy(), img_bgr.copy(), np.zeros_like(img_bgr)

    masks = res.masks.data.cpu().numpy()
    classes = res.masks.cls.cpu().numpy() if hasattr(res.masks, "cls") \
              else res.boxes.cls.cpu().numpy()
    h_m, w_m = masks.shape[1:]

    rail_union = np.zeros((h_m, w_m), bool)
    for m, c in zip(masks, classes):
        if int(c) == RAIL_ID:
            rail_union |= m.astype(bool)

    rail_mask = rail_union
    if rail_mask.shape != (H, W):
        rail_mask = cv2.resize(rail_mask.astype(np.uint8), (W, H), interpolation=cv2.INTER_NEAREST).astype(bool)

    green = highlight_rails_mask_only_fast(img_bgr, rail_mask,
                                           TARGET_COLORS_RGB,
                                           TOLERANCE,
                                           MIN_REGION_SIZE,
                                           MIN_REGION_HEIGHT)
    red = rail_mask & ~green
    score = red_vs_green_score(red, green, HEAT_BLUR_KSIZE)

    top_ex = int(H * EXCLUDE_TOP_FRAC)
    bot_ex = int(H * EXCLUDE_BOTTOM_FRAC)
    score[:top_ex, :] = 0
    if bot_ex: score[H-bot_ex:, :] = 0

    dark = (score >= RED_SCORE_THRESH).astype(np.uint8)
    dark = cv2.morphologyEx(dark, cv2.MORPH_OPEN,
                            cv2.getStructuringElement(cv2.MORPH_RECT, (5,9)),
                            iterations=1)

    left = img_bgr.copy()
    overlay = left.copy()
    for m, c in zip(masks, classes):
        cid = int(c)
        if cid == RAIL_ID:
            continue
        mask_full = m
        if mask_full.shape != (H, W):
            mask_full = cv2.resize(mask_full.astype(np.uint8), (W, H), interpolation=cv2.INTER_NEAREST).astype(bool)

        label = obstacle_classes.get(cid, f"CLASS {cid}")
        colour = CLASS_COLOURS.get(cid, (255, 255, 255))
        overlay[mask_full] = colour

        ys, xs = np.where(mask_full)
        if len(xs):
            x_c, y_c = int(xs.mean()), int(ys.mean())
            cv2.putText(overlay, label, (x_c - 40, y_c),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 2, cv2.LINE_AA)
            cv2.putText(overlay, label, (x_c - 40, y_c),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)

    left = cv2.addWeighted(overlay, 0.6, left, 0.4, 0)

    n_lbl, lbl_mat, stats, _ = cv2.connectedComponentsWithStats(dark, 8)
    for lbl in range(1, n_lbl):
        if stats[lbl, cv2.CC_STAT_AREA] < MIN_DARK_RED_AREA:
            continue
        ys, xs = np.where(lbl_mat == lbl)
        y_top = ys.min()
        x_mid = int(xs[ys == y_top].mean())
        draw_triangle(left, x_mid, y_top)

    right = img_bgr.copy()
    rails_tinted = right.copy()
    if rail_mask.shape != right.shape[:2]:
        rail_mask = rail_mask.astype(np.uint8)
    rails_tinted[rail_mask] = (0,0,255)
    right = cv2.addWeighted(rails_tinted, ALPHA, right, 1-ALPHA, 0)
    right[green] = (0,255,0)

    heat_col = cv2.applyColorMap(score, cv2.COLORMAP_JET)
    heat_col = cv2.resize(heat_col, (left.shape[1]+right.shape[1], H), interpolation=cv2.INTER_LINEAR)

    return left, right, heat_col

# ------------------------------------------------------------------
if __name__ == "__main__":
    folder = f"{home}/SubwaySurfers/train_screenshots"
    paths = sorted(glob.glob(os.path.join(folder, "frame_*.jpg")))
    if not paths:
        print("No frames found", file=sys.stderr); sys.exit(1)

    for p in paths:
        frame = cv2.imread(p);  t0 = time.time()
        left, right, heat = process_frame(frame)
        top = np.hstack((left, right))
        canvas = np.vstack((top, heat))
        cv2.imshow("Left: Labelled masks | Right: Rails tinted | Bottom: heat-map", canvas)
        print(f"{os.path.basename(p):>20s}   proc {1000*(time.time()-t0):5.1f} ms")
        key = cv2.waitKey(0) & 0xFF
        if key in (ord('q'), 27):
            break
    cv2.destroyAllWindows()


In [ ]:
#BLUESTACKS dimensions uses mobile resolution vertical frames

In [ ]:
#!/usr/bin/env python3
"""
Visualise *all* segmentation masks with labelled overlays.

• Rails are tinted red (right) and used for the heat-map + purple-triangle.
• Other masks are labelled and coloured on the original (left).
• Bottom panel shows the heat-map.

Controls:
  SPACE: next frame   |   q / ESC: quit

Notes:
- Uses a responsive event loop (waitKey(1)) so keys register immediately.
- Canvas is auto-rescaled to avoid huge window draw latency.
"""

import os, glob, sys, time
import cv2, torch, numpy as np
from pathlib import Path
from ultralytics import YOLO

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40

TARGET_COLORS_RGB  = [(119,104,67), (81,42,45)]
TOLERANCE          = 20.0
MIN_REGION_SIZE    = 30
MIN_REGION_HEIGHT  = 150

HEAT_BLUR_KSIZE     = 51
RED_SCORE_THRESH    = 220
EXCLUDE_TOP_FRAC    = 0.40
EXCLUDE_BOTTOM_FRAC = 0.15
MIN_DARK_RED_AREA   = 1200
TRI_SIZE_PX         = 18
PURPLE              = (255, 0, 255)

# UI / display constraints
WIN_NAME        = "Left: labels | Right: rails | Bottom: heat-map (SPACE=next, q=quit)"
MAX_DISPLAY_W   = 1600   # shrink canvas if wider than this to avoid huge blits
TEXT_CLR        = (255, 255, 255)

obstacle_classes = {
    0: "BOOTS", 1: "GREYTRAIN", 2: "HIGHBARRIER1", 3: "JUMP", 4: "LOWBARRIER1",
    5: "LOWBARRIER2", 6: "ORANGETRAIN", 7: "PILLAR", 8: "RAMP", 9: "RAILS",
    10: "SIDEWALK", 11: "YELLOWTRAIN"
}
CLASS_COLOURS = {
    0: (255,255,0), 1: (192,192,192), 2: (0,128,255), 3: (0,255,0),
    4: (255,0,255), 5: (0,255,255), 6: (255,128,0), 7: (128,0,255),
    8: (0,0,128), 10: (128,128,0), 11: (255,255,102)
}

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# Optional: reduce OpenCV CPU thread contention with PyTorch
try:
    cv2.setNumThreads(1)
except Exception:
    pass

model = YOLO(weights)
try:
    model.fuse()
except Exception:
    pass

# Warm-up so the first real frame isn't laggy
_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(_dummy, task="segment", imgsz=IMG_SIZE, device=device,
                  conf=CONF, iou=IOU, verbose=False, half=half)

# ------------------------------------------------------------------
def highlight_rails_mask_only_fast(img_bgr, rail_mask, target_colors_rgb,
                                   tolerance=30.0,
                                   min_region_size=50,
                                   min_region_height=150):
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max()+1
    x0, x1 = xs.min(), xs.max()+1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    targets_bgr = np.array([(r,g,b)[::-1] for r,g,b in target_colors_rgb],
                           dtype=np.float32)
    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - targets_bgr[None, None, :, :]
    dist2 = np.sum(diff*diff, axis=-1)
    colour_hit = np.any(dist2 <= tolerance**2, axis=-1)

    combined = colour_hit & mask_roi
    comp = combined.astype(np.uint8)
    n, lbls, stats, _ = cv2.connectedComponentsWithStats(comp, 8)

    good = np.zeros_like(combined)
    for lbl in range(1, n):
        area, h = stats[lbl, cv2.CC_STAT_AREA], stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and h >= min_region_height:
            good[lbls == lbl] = True

    filtered_full[y0:y1, x0:x1] = good
    return filtered_full


def red_vs_green_score(red_mask, green_mask, blur_ksize=51):
    k = (blur_ksize, blur_ksize)
    r = cv2.blur(red_mask.astype(np.float32), k)
    g = cv2.blur(green_mask.astype(np.float32), k)
    diff = r - g
    amax = float(np.max(np.abs(diff))) + 1e-6
    norm = (diff / (2*amax) + 0.5)
    return np.clip(norm * 255, 0, 255).astype(np.uint8)


def draw_triangle(img, x, y, size=TRI_SIZE_PX, colour=PURPLE):
    h = int(size * 1.2)
    pts = np.array([[x, y], [x-size, y+h], [x+size, y+h]], np.int32)
    cv2.fillConvexPoly(img, pts, colour)
    cv2.polylines(img, [pts.reshape(-1,1,2)], True, (0,0,0), 1, cv2.LINE_AA)

# ------------------------------------------------------------------
def process_frame(img_bgr):
    """Returns (left, right, heat_col) – all uint8 BGR images."""
    res = model.predict(img_bgr, task="segment", imgsz=IMG_SIZE,
                        device=device, conf=CONF, iou=IOU,
                        max_det=30, verbose=False, half=half)[0]

    H, W = img_bgr.shape[:2]
    if res.masks is None:
        blank_heat = np.zeros((H, W), np.uint8)
        return img_bgr.copy(), img_bgr.copy(), cv2.applyColorMap(blank_heat, cv2.COLORMAP_JET)

    masks   = res.masks.data.cpu().numpy()
    classes = res.masks.cls.cpu().numpy() if hasattr(res.masks, "cls") \
              else res.boxes.cls.cpu().numpy()
    h_m, w_m = masks.shape[1:]

    rail_union = np.zeros((h_m, w_m), bool)
    for m, c in zip(masks, classes):
        if int(c) == RAIL_ID:
            rail_union |= m.astype(bool)

    rail_mask = rail_union
    if rail_mask.shape != (H, W):
        rail_mask = cv2.resize(rail_mask.astype(np.uint8), (W, H),
                               interpolation=cv2.INTER_NEAREST).astype(bool)

    green = highlight_rails_mask_only_fast(img_bgr, rail_mask,
                                           TARGET_COLORS_RGB, TOLERANCE,
                                           MIN_REGION_SIZE, MIN_REGION_HEIGHT)
    red   = rail_mask & ~green
    score = red_vs_green_score(red, green, HEAT_BLUR_KSIZE)

    top_ex = int(H * EXCLUDE_TOP_FRAC)
    bot_ex = int(H * EXCLUDE_BOTTOM_FRAC)
    score[:top_ex, :] = 0
    if bot_ex: score[H-bot_ex:, :] = 0

    dark = (score >= RED_SCORE_THRESH).astype(np.uint8)
    dark = cv2.morphologyEx(
        dark, cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_RECT, (5, 9)),
        iterations=1
    )

    # left: labels overlaid
    left    = img_bgr.copy()
    overlay = left.copy()
    for m, c in zip(masks, classes):
        cid = int(c)
        if cid == RAIL_ID:
            continue
        mask_full = m
        if mask_full.shape != (H, W):
            mask_full = cv2.resize(mask_full.astype(np.uint8), (W, H),
                                   interpolation=cv2.INTER_NEAREST).astype(bool)
        label  = obstacle_classes.get(cid, f"CLASS {cid}")
        colour = CLASS_COLOURS.get(cid, (255, 255, 255))
        overlay[mask_full] = colour

        ys, xs = np.where(mask_full)
        if len(xs):
            x_c, y_c = int(xs.mean()), int(ys.mean())
            cv2.putText(overlay, label, (x_c - 40, y_c),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 2, cv2.LINE_AA)
            cv2.putText(overlay, label, (x_c - 40, y_c),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)
    left = cv2.addWeighted(overlay, 0.6, left, 0.4, 0)

    # purple triangle warnings
    n_lbl, lbl_mat, stats, _ = cv2.connectedComponentsWithStats(dark, 8)
    for lbl in range(1, n_lbl):
        if stats[lbl, cv2.CC_STAT_AREA] < MIN_DARK_RED_AREA: continue
        ys, xs = np.where(lbl_mat == lbl)
        y_top, x_mid = ys.min(), int(xs[ys == ys.min()].mean())
        draw_triangle(left, x_mid, y_top)

    # right: rails tinted
    right = img_bgr.copy()
    rails_tinted = right.copy()
    rails_tinted[rail_mask] = (0,0,255)
    right = cv2.addWeighted(rails_tinted, ALPHA, right, 1-ALPHA, 0)
    right[green] = (0,255,0)

    # bottom: heat map (width = left+right, height = H)
    heat_col = cv2.applyColorMap(score, cv2.COLORMAP_JET)
    heat_col = cv2.resize(heat_col, (left.shape[1] + right.shape[1], H),
                          interpolation=cv2.INTER_LINEAR)

    return left, right, heat_col

def assemble_canvas(left, right, heat):
    top    = np.hstack((left, right))
    canvas = np.vstack((top, heat))
    return canvas

def maybe_downscale(img, max_w=MAX_DISPLAY_W):
    h, w = img.shape[:2]
    if w <= max_w:
        return img
    scale = max_w / float(w)
    new_size = (int(w*scale), int(h*scale))
    return cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)

# ------------------------------------------------------------------
if __name__ == "__main__":
    folder = Path.home() / "Documents" / "GitHub" / "Ai-plays-SubwaySurfers" / "frames"
    if not folder.is_dir():
        print(f"frames folder not found: {folder}", file=sys.stderr); sys.exit(1)

    paths = sorted(
        glob.glob(str(folder / "frame_*.jpg")) +
        glob.glob(str(folder / "frame_*.png")) +
        glob.glob(str(folder / "*.jpg")) +
        glob.glob(str(folder / "*.png"))
    )
    if not paths:
        print(f"No frame images found in {folder}", file=sys.stderr); sys.exit(1)

    cv2.namedWindow(WIN_NAME, cv2.WINDOW_NORMAL)

    i = 0
    n = len(paths)
    while i < n:
        p = paths[i]
        frame = cv2.imread(p)
        if frame is None:
            print(f"[WARN] unreadable image: {p}")
            i += 1
            continue

        try:
            t0 = time.perf_counter()
            left, right, heat = process_frame(frame)
            proc_ms = (time.perf_counter() - t0) * 1000.0
            canvas = assemble_canvas(left, right, heat)
            canvas = maybe_downscale(canvas, MAX_DISPLAY_W)

            # On-screen text for processing time
            cv2.putText(canvas, f"{os.path.basename(p)}  |  proc: {proc_ms:.1f} ms",
                        (12, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 3, cv2.LINE_AA)
            cv2.putText(canvas, f"{os.path.basename(p)}  |  proc: {proc_ms:.1f} ms",
                        (12, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.8, TEXT_CLR, 1, cv2.LINE_AA)
            cv2.putText(canvas, "SPACE: next   q/ESC: quit",
                        (12, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 3, cv2.LINE_AA)
            cv2.putText(canvas, "SPACE: next   q/ESC: quit",
                        (12, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, TEXT_CLR, 1, cv2.LINE_AA)

        except Exception as e:
            print(f"[WARN] error on {os.path.basename(p)}: {e}")
            i += 1
            continue

        # Responsive event loop: keep pumping events so SPACE is captured
        while True:
            cv2.imshow(WIN_NAME, canvas)
            key = cv2.waitKey(1) & 0xFF

            # Handle window close (on some platforms returns -1 repeatedly after close)
            if cv2.getWindowProperty(WIN_NAME, cv2.WND_PROP_VISIBLE) < 1:
                cv2.destroyAllWindows()
                sys.exit(0)

            if key == 32:          # SPACE → next
                i += 1
                break
            elif key in (ord('q'), 27):  # q or ESC
                cv2.destroyAllWindows()
                sys.exit(0)
            # else: loop continues; window remains responsive

    cv2.destroyAllWindows()


In [ ]:
#!/usr/bin/env python3
"""
Speed-test: run the full segmentation/overlay pipeline on every image in
~/Documents/GitHub/Ai-plays-SubwaySurfers/frames and print a timing
summary at the end.

Changes in this version
────────────────────────
• The first WARMUP_FRAMES (default 10) are *executed* but **not timed**,
  so model/kernel warm-up and disk cache effects don’t skew results.
• Everything else is unchanged: no GUI overhead, concise summary.
"""

import os, sys, glob, time, statistics
from pathlib import Path

import cv2
import numpy as np
import torch
from ultralytics import YOLO

# ───────────────────────────────────────────────────────── Config ──
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40

TARGET_COLORS_RGB  = [(119,104,67), (81,42,45)]
TOLERANCE          = 20.0
MIN_REGION_SIZE    = 30
MIN_REGION_HEIGHT  = 150

HEAT_BLUR_KSIZE     = 51
RED_SCORE_THRESH    = 220
EXCLUDE_TOP_FRAC    = 0.40
EXCLUDE_BOTTOM_FRAC = 0.15
MIN_DARK_RED_AREA   = 1200
TRI_SIZE_PX         = 18
PURPLE              = (255, 0, 255)

WARMUP_FRAMES       = 10   # how many frames to skip from timing stats

obstacle_classes = {
    0: "BOOTS", 1: "GREYTRAIN", 2: "HIGHBARRIER1", 3: "JUMP", 4: "LOWBARRIER1",
    5: "LOWBARRIER2", 6: "ORANGETRAIN", 7: "PILLAR", 8: "RAMP", 9: "RAILS",
    10: "SIDEWALK", 11: "YELLOWTRAIN"
}
CLASS_COLOURS = {
    0: (255,255,0), 1: (192,192,192), 2: (0,128,255), 3: (0,255,0),
    4: (255,0,255), 5: (0,255,255), 6: (255,128,0), 7: (128,0,255),
    8: (0,0,128), 10: (128,128,0), 11: (255,255,102)
}

# ─────────────────────────────────────── Device / model init ─
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

model = YOLO(weights)
try: model.fuse()
except Exception: pass
model.predict(
    np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8),
    task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, verbose=False, half=half
)

# ───────────────────────────── Helper processing functions ─
def highlight_rails_mask_only_fast(img_bgr, rail_mask, target_colors_rgb,
                                   tolerance=30.0,
                                   min_region_size=50,
                                   min_region_height=150):
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max()+1
    x0, x1 = xs.min(), xs.max()+1
    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    targets_bgr = np.array([(r,g,b)[::-1] for r,g,b in target_colors_rgb],
                           dtype=np.float32)
    diff  = img_roi.astype(np.float32)[:, :, None, :] - targets_bgr[None, None]
    dist2 = np.sum(diff*diff, axis=-1)
    colour_hit = np.any(dist2 <= tolerance**2, axis=-1)

    combined = colour_hit & mask_roi
    comp = combined.astype(np.uint8)
    n, lbls, stats, _ = cv2.connectedComponentsWithStats(comp, 8)

    good = np.zeros_like(combined)
    for lbl in range(1, n):
        area, h = stats[lbl, cv2.CC_STAT_AREA], stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and h >= min_region_height:
            good[lbls == lbl] = True

    filtered_full[y0:y1, x0:x1] = good
    return filtered_full


def red_vs_green_score(red_mask, green_mask, blur_ksize=51):
    k = (blur_ksize, blur_ksize)
    diff = (cv2.blur(red_mask.astype(np.float32), k) -
            cv2.blur(green_mask.astype(np.float32), k))
    norm = (diff / (2*(np.max(np.abs(diff))+1e-6)) + 0.5)
    return np.clip(norm * 255, 0, 255).astype(np.uint8)


def process_frame(img_bgr):
    res = model.predict(img_bgr, task="segment", imgsz=IMG_SIZE,
                        device=device, conf=CONF, iou=IOU,
                        max_det=30, verbose=False, half=half)[0]
    if res.masks is None:
        return

    masks   = res.masks.data.cpu().numpy()
    classes = res.masks.cls.cpu().numpy() if hasattr(res.masks, "cls") \
              else res.boxes.cls.cpu().numpy()

    H, W = img_bgr.shape[:2]
    rail_union = np.zeros(masks.shape[1:], bool)
    for m, c in zip(masks, classes):
        if int(c) == RAIL_ID:
            rail_union |= m.astype(bool)

    rail_mask = cv2.resize(rail_union.astype(np.uint8), (W, H),
                           interpolation=cv2.INTER_NEAREST).astype(bool)

    green = highlight_rails_mask_only_fast(img_bgr, rail_mask,
                                           TARGET_COLORS_RGB, TOLERANCE,
                                           MIN_REGION_SIZE, MIN_REGION_HEIGHT)
    red   = rail_mask & ~green
    score = red_vs_green_score(red, green, HEAT_BLUR_KSIZE)

    # exclude top/bottom
    score[:int(H*EXCLUDE_TOP_FRAC), :] = 0
    score[H-int(H*EXCLUDE_BOTTOM_FRAC):, :] = 0

    dark = (score >= RED_SCORE_THRESH).astype(np.uint8)
    cv2.morphologyEx(
        dark, cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_RECT, (5, 9)),
        dst=dark, iterations=1
    )
    # outputs are not displayed; function exists for timing only.

# ─────────────────────────────────────────────────────── Main test ──
if __name__ == "__main__":
    folder = (Path.home() / "Documents" / "GitHub" /
              "Ai-plays-SubwaySurfers" / "frames")

    if not folder.is_dir():
        print(f"[ERROR] frames folder not found: {folder}", file=sys.stderr)
        sys.exit(1)

    paths = sorted(
        glob.glob(str(folder / "frame_*.jpg")) +
        glob.glob(str(folder / "frame_*.png")) +
        glob.glob(str(folder / "*.jpg")) +
        glob.glob(str(folder / "*.png"))
    )
    if not paths:
        print(f"[ERROR] No frame images found in {folder}", file=sys.stderr)
        sys.exit(1)

    times = []
    t_start = time.perf_counter()

    for idx, p in enumerate(paths):
        frame = cv2.imread(p)

        if idx < WARMUP_FRAMES:
            # Warm-up: run but don't time
            process_frame(frame)
            continue

        t0 = time.perf_counter()
        process_frame(frame)
        times.append(time.perf_counter() - t0)
        print(time.perf_counter() - t0)

    total_time = time.perf_counter() - t_start
    timed_frames = len(times)

    # ─────────────────────────────── Summary ─
    ms = [t * 1_000 for t in times]
    if timed_frames == 0:
        print("\n[WARNING] fewer frames than WARMUP_FRAMES; nothing timed.")
        sys.exit(0)

    print("\n───── Speed-test summary (warm-up skipped) ─────")
    print(f"Total frames           : {len(paths)}")
    print(f"Warm-up frames ignored : {WARMUP_FRAMES}")
    print(f"Frames timed           : {timed_frames}")
    print(f"Total wall-clock time  : {total_time:,.2f} s")
    print(f"Average / frame        : {statistics.mean(ms):,.2f} ms"
          f"  ({1_000/statistics.mean(ms):,.2f} FPS)")
    print(f"Median                 : {statistics.median(ms):,.2f} ms")
    print(f"Fastest                : {min(ms):,.2f} ms")
    print(f"Slowest                : {max(ms):,.2f} ms")
    print("────────────────────────────────────────────────")


In [ ]:
#Filtering out smaller red regions to minimise misreads

In [ ]:
#!/usr/bin/env python3
"""
Visualise *all* segmentation masks with labelled overlays.

• Rails are tinted red (right) and used for the heat-map + purple-triangle.
• Other masks are labelled and coloured on the original (left).
• Bottom panel shows the heat-map.

Controls:
  SPACE: next frame   |   q / ESC: quit

Loads images from:
    ~/Documents/GitHub/Ai-plays-SubwaySurfers/frames

Change requested:
  Before plotting a purple triangle, require that the component's dark-red
  area is at least 15% of the *total* dark-red area observed for the frame.
  Implemented with negligible overhead and no other logic changes.
"""

import os, glob, sys, time
import cv2, torch, numpy as np
from pathlib import Path
from ultralytics import YOLO

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40

TARGET_COLORS_RGB  = [(119,104,67), (81,42,45)]
TOLERANCE          = 20.0
MIN_REGION_SIZE    = 30
MIN_REGION_HEIGHT  = 150

HEAT_BLUR_KSIZE     = 51
RED_SCORE_THRESH    = 220
EXCLUDE_TOP_FRAC    = 0.40
EXCLUDE_BOTTOM_FRAC = 0.15
MIN_DARK_RED_AREA   = 1200
TRI_SIZE_PX         = 18
PURPLE              = (255, 0, 255)

# New: minimum fraction of total dark-red area a blob must contribute
MIN_DARK_FRACTION   = 0.15  # 15%

# UI / display constraints
WIN_NAME        = "Left: labels | Right: rails | Bottom: heat-map (SPACE=next, q=quit)"
MAX_DISPLAY_W   = 1600
TEXT_CLR        = (255, 255, 255)

obstacle_classes = {
    0: "BOOTS", 1: "GREYTRAIN", 2: "HIGHBARRIER1", 3: "JUMP", 4: "LOWBARRIER1",
    5: "LOWBARRIER2", 6: "ORANGETRAIN", 7: "PILLAR", 8: "RAMP", 9: "RAILS",
    10: "SIDEWALK", 11: "YELLOWTRAIN"
}
CLASS_COLOURS = {
    0: (255,255,0), 1: (192,192,192), 2: (0,128,255), 3: (0,255,0),
    4: (255,0,255), 5: (0,255,255), 6: (255,128,0), 7: (128,0,255),
    8: (0,0,128), 10: (128,128,0), 11: (255,255,102)
}

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# Optional: reduce OpenCV CPU thread contention with PyTorch
try:
    cv2.setNumThreads(1)
except Exception:
    pass

model = YOLO(weights)
try: model.fuse()
except Exception: pass

# warm up once so the first frame isn't laggy
_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(_dummy, task="segment", imgsz=IMG_SIZE, device=device,
                  conf=CONF, iou=IOU, verbose=False, half=half)

# ------------------------------------------------------------------
def highlight_rails_mask_only_fast(img_bgr, rail_mask, target_colors_rgb,
                                   tolerance=30.0,
                                   min_region_size=50,
                                   min_region_height=150):
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max()+1
    x0, x1 = xs.min(), xs.max()+1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    targets_bgr = np.array([(r,g,b)[::-1] for r,g,b in target_colors_rgb],
                           dtype=np.float32)
    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - targets_bgr[None, None, :, :]
    dist2 = np.sum(diff*diff, axis=-1)
    colour_hit = np.any(dist2 <= tolerance**2, axis=-1)

    combined = colour_hit & mask_roi
    comp = combined.astype(np.uint8)
    n, lbls, stats, _ = cv2.connectedComponentsWithStats(comp, 8)

    good = np.zeros_like(combined)
    for lbl in range(1, n):
        area, h = stats[lbl, cv2.CC_STAT_AREA], stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and h >= min_region_height:
            good[lbls == lbl] = True

    filtered_full[y0:y1, x0:x1] = good
    return filtered_full


def red_vs_green_score(red_mask, green_mask, blur_ksize=51):
    k = (blur_ksize, blur_ksize)
    r = cv2.blur(red_mask.astype(np.float32), k)
    g = cv2.blur(green_mask.astype(np.float32), k)
    diff = r - g
    amax = float(np.max(np.abs(diff))) + 1e-6
    norm = (diff / (2*amax) + 0.5)
    return np.clip(norm * 255, 0, 255).astype(np.uint8)


def draw_triangle(img, x, y, size=TRI_SIZE_PX, colour=PURPLE):
    h = int(size * 1.2)
    pts = np.array([[x, y], [x-size, y+h], [x+size, y+h]], np.int32)
    cv2.fillConvexPoly(img, pts, colour)
    cv2.polylines(img, [pts.reshape(-1,1,2)], True, (0,0,0), 1, cv2.LINE_AA)

# ------------------------------------------------------------------
def process_frame(img_bgr):
    """Returns (left, right, heat_col) – all uint8 BGR images."""
    res = model.predict(img_bgr, task="segment", imgsz=IMG_SIZE,
                        device=device, conf=CONF, iou=IOU,
                        max_det=30, verbose=False, half=half)[0]

    H, W = img_bgr.shape[:2]
    if res.masks is None:
        blank_heat = np.zeros((H, W), np.uint8)
        return img_bgr.copy(), img_bgr.copy(), cv2.applyColorMap(blank_heat, cv2.COLORMAP_JET)

    masks   = res.masks.data.cpu().numpy()
    classes = res.masks.cls.cpu().numpy() if hasattr(res.masks, "cls") \
              else res.boxes.cls.cpu().numpy()
    h_m, w_m = masks.shape[1:]

    rail_union = np.zeros((h_m, w_m), bool)
    for m, c in zip(masks, classes):
        if int(c) == RAIL_ID:
            rail_union |= m.astype(bool)

    rail_mask = rail_union
    if rail_mask.shape != (H, W):
        rail_mask = cv2.resize(rail_mask.astype(np.uint8), (W, H),
                               interpolation=cv2.INTER_NEAREST).astype(bool)

    green = highlight_rails_mask_only_fast(img_bgr, rail_mask,
                                           TARGET_COLORS_RGB, TOLERANCE,
                                           MIN_REGION_SIZE, MIN_REGION_HEIGHT)
    red   = rail_mask & ~green
    score = red_vs_green_score(red, green, HEAT_BLUR_KSIZE)

    top_ex = int(H * EXCLUDE_TOP_FRAC)
    bot_ex = int(H * EXCLUDE_BOTTOM_FRAC)
    score[:top_ex, :] = 0
    if bot_ex: score[H-bot_ex:, :] = 0

    dark = (score >= RED_SCORE_THRESH).astype(np.uint8)
    dark = cv2.morphologyEx(
        dark, cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_RECT, (5, 9)),
        iterations=1
    )

    # ================== New tiny-overhead filter ==================
    # Require a component to be at least MIN_DARK_FRACTION of total dark area.
    total_dark_area = int(dark.sum())
    frac_area_thresh = int(np.ceil(MIN_DARK_FRACTION * total_dark_area))
    # =============================================================

    # left: labels overlaid
    left    = img_bgr.copy()
    overlay = left.copy()
    for m, c in zip(masks, classes):
        cid = int(c)
        if cid == RAIL_ID:
            continue
        mask_full = m
        if mask_full.shape != (H, W):
            mask_full = cv2.resize(mask_full.astype(np.uint8), (W, H),
                                   interpolation=cv2.INTER_NEAREST).astype(bool)
        label  = obstacle_classes.get(cid, f"CLASS {cid}")
        colour = CLASS_COLOURS.get(cid, (255, 255, 255))
        overlay[mask_full] = colour

        ys, xs = np.where(mask_full)
        if len(xs):
            x_c, y_c = int(xs.mean()), int(ys.mean())
            cv2.putText(overlay, label, (x_c - 40, y_c),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 2, cv2.LINE_AA)
            cv2.putText(overlay, label, (x_c - 40, y_c),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)
    left = cv2.addWeighted(overlay, 0.6, left, 0.4, 0)

    # purple triangle warnings
    n_lbl, lbl_mat, stats, _ = cv2.connectedComponentsWithStats(dark, 8)
    for lbl in range(1, n_lbl):
        area = stats[lbl, cv2.CC_STAT_AREA]
        if area < MIN_DARK_RED_AREA or area < frac_area_thresh:
            continue
        ys, xs = np.where(lbl_mat == lbl)
        y_top, x_mid = ys.min(), int(xs[ys == ys.min()].mean())
        draw_triangle(left, x_mid, y_top)

    # right: rails tinted
    right = img_bgr.copy()
    rails_tinted = right.copy()
    rails_tinted[rail_mask] = (0,0,255)
    right = cv2.addWeighted(rails_tinted, ALPHA, right, 1-ALPHA, 0)
    right[green] = (0,255,0)

    # bottom: heat map (width = left+right, height = H)
    heat_col = cv2.applyColorMap(score, cv2.COLORMAP_JET)
    heat_col = cv2.resize(heat_col, (left.shape[1] + right.shape[1], H),
                          interpolation=cv2.INTER_LINEAR)

    return left, right, heat_col

def assemble_canvas(left, right, heat):
    top    = np.hstack((left, right))
    canvas = np.vstack((top, heat))
    return canvas

def maybe_downscale(img, max_w=MAX_DISPLAY_W):
    h, w = img.shape[:2]
    if w <= max_w:
        return img
    scale = max_w / float(w)
    new_size = (int(w*scale), int(h*scale))
    return cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)

# ------------------------------------------------------------------
if __name__ == "__main__":
    folder = Path.home() / "Documents" / "GitHub" / "Ai-plays-SubwaySurfers" / "frames"
    if not folder.is_dir():
        print(f"frames folder not found: {folder}", file=sys.stderr); sys.exit(1)

    paths = sorted(
        glob.glob(str(folder / "frame_*.jpg")) +
        glob.glob(str(folder / "frame_*.png")) +
        glob.glob(str(folder / "*.jpg")) +
        glob.glob(str(folder / "*.png"))
    )
    if not paths:
        print(f"No frame images found in {folder}", file=sys.stderr); sys.exit(1)

    cv2.namedWindow(WIN_NAME, cv2.WINDOW_NORMAL)

    i = 0
    n = len(paths)
    while i < n:
        p = paths[i]
        frame = cv2.imread(p)
        if frame is None:
            print(f"[WARN] unreadable image: {p}")
            i += 1
            continue

        try:
            t0 = time.perf_counter()
            left, right, heat = process_frame(frame)
            proc_ms = (time.perf_counter() - t0) * 1000.0
            canvas = assemble_canvas(left, right, heat)
            canvas = maybe_downscale(canvas, MAX_DISPLAY_W)

            # On-screen text for processing time
            cv2.putText(canvas, f"{os.path.basename(p)}  |  proc: {proc_ms:.1f} ms",
                        (12, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 3, cv2.LINE_AA)
            cv2.putText(canvas, f"{os.path.basename(p)}  |  proc: {proc_ms:.1f} ms",
                        (12, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.8, TEXT_CLR, 1, cv2.LINE_AA)
            cv2.putText(canvas, "SPACE: next   q/ESC: quit",
                        (12, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 3, cv2.LINE_AA)
            cv2.putText(canvas, "SPACE: next   q/ESC: quit",
                        (12, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, TEXT_CLR, 1, cv2.LINE_AA)

        except Exception as e:
            print(f"[WARN] error on {os.path.basename(p)}: {e}")
            i += 1
            continue

        # Responsive event loop
        while True:
            cv2.imshow(WIN_NAME, canvas)
            key = cv2.waitKey(1) & 0xFF

            # Handle window close
            if cv2.getWindowProperty(WIN_NAME, cv2.WND_PROP_VISIBLE) < 1:
                cv2.destroyAllWindows()
                sys.exit(0)

            if key == 32:          # SPACE → next
                i += 1
                break
            elif key in (ord('q'), 27):  # q or ESC
                cv2.destroyAllWindows()
                sys.exit(0)

    cv2.destroyAllWindows()


In [ ]:
#!/usr/bin/env python3
"""
Visualise *all* segmentation masks with labelled overlays + Jake bbox.

Additions (no double work, minimal overhead):
• Runs RF-DETR once per frame to detect Jake (green rectangle).
• Finds the YOLO mask with the largest overlap under Jake's bbox and
  tints that object PINK so you can see what he's under.
• Reuses the same YOLO segmentation output for everything (no duplicate inference).
• Keeps the responsive SPACE-next UI and the purple-triangle logic (with 15% area filter).

Controls:
  SPACE: next frame   |   q / ESC: quit
"""

import os, glob, sys, time
# ── Enable CPU fallback before importing torch (for MPS gaps) ────────────────
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import cv2
import numpy as np
import torch
import torch.nn.functional as F
from pathlib import Path
from PIL import Image
from ultralytics import YOLO
from rfdetr import RFDETRBase

# ── Monkey-patch interpolate to disable antialias (avoids unsupported MPS op) ─
_original_interpolate = F.interpolate
def _interpolate_no_antialias(input, *args, **kwargs):
    kwargs['antialias'] = False
    return _original_interpolate(input, *args, **kwargs)
F.interpolate = _interpolate_no_antialias

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
yolo_weights = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
jake_weights = f"{home}/downloads/weightsjake.pt"

RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40

TARGET_COLORS_RGB  = [(119,104,67), (81,42,45)]
TOLERANCE          = 20.0
MIN_REGION_SIZE    = 30
MIN_REGION_HEIGHT  = 150

HEAT_BLUR_KSIZE     = 51
RED_SCORE_THRESH    = 220
EXCLUDE_TOP_FRAC    = 0.40
EXCLUDE_BOTTOM_FRAC = 0.15
MIN_DARK_RED_AREA   = 1200
TRI_SIZE_PX         = 18
PURPLE              = (255, 0, 255)

# Purple-triangle extra filter: blob must be ≥ this fraction of total dark area
MIN_DARK_FRACTION   = 0.15  # 15%

# UI / display constraints
WIN_NAME        = "Left: labels | Right: rails | Bottom: heat (SPACE=next, q=quit)"
MAX_DISPLAY_W   = 1600
TEXT_CLR        = (255, 255, 255)
JAKE_BOX_CLR    = (0, 255, 0)   # green bbox around Jake
UNDER_TINT_BGR  = (255, 0, 255) # pink/magenta tint for object Jake is under

obstacle_classes = {
    0: "BOOTS", 1: "GREYTRAIN", 2: "HIGHBARRIER1", 3: "JUMP", 4: "LOWBARRIER1",
    5: "LOWBARRIER2", 6: "ORANGETRAIN", 7: "PILLAR", 8: "RAMP", 9: "RAILS",
    10: "SIDEWALK", 11: "YELLOWTRAIN"
}
CLASS_COLOURS = {
    0: (255,255,0), 1: (192,192,192), 2: (0,128,255), 3: (0,255,0),
    4: (255,0,255), 5: (0,255,255), 6: (255,128,0), 7: (128,0,255),
    8: (0,0,128), 10: (128,128,0), 11: (255,255,102)
}

# =======================
# Device / precision
# =======================
# YOLO device: CUDA > MPS > CPU
if torch.cuda.is_available():
    yolo_device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    yolo_device, half = "mps", False
else:
    yolo_device, half = "cpu", False

# RF-DETR device: prefer MPS if present, else CPU (it throws NotImplementedError for some ops)
det_device = "mps" if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available() else "cpu"

# Optional: reduce OpenCV CPU thread contention with PyTorch
try:
    cv2.setNumThreads(1)
except Exception:
    pass

# =======================
# Load models + warmup
# =======================
yolo_model = YOLO(yolo_weights)
try: yolo_model.fuse()
except Exception: pass

jake_model = RFDETRBase(pretrain_weights=jake_weights, num_classes=3)

# Warm-up YOLO so first frame isn't laggy
_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = yolo_model.predict(_dummy, task="segment", imgsz=IMG_SIZE,
                       device=yolo_device, conf=CONF, iou=IOU,
                       verbose=False, half=half)

# ------------------------------------------------------------------
def highlight_rails_mask_only_fast(img_bgr, rail_mask, target_colors_rgb,
                                   tolerance=30.0,
                                   min_region_size=50,
                                   min_region_height=150):
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max()+1
    x0, x1 = xs.min(), xs.max()+1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    targets_bgr = np.array([(r,g,b)[::-1] for r,g,b in target_colors_rgb],
                           dtype=np.float32)
    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - targets_bgr[None, None, :, :]
    dist2 = np.sum(diff*diff, axis=-1)
    colour_hit = np.any(dist2 <= tolerance**2, axis=-1)

    combined = colour_hit & mask_roi
    comp = combined.astype(np.uint8)
    n, lbls, stats, _ = cv2.connectedComponentsWithStats(comp, 8)

    good = np.zeros_like(combined)
    for lbl in range(1, n):
        area, h = stats[lbl, cv2.CC_STAT_AREA], stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and h >= min_region_height:
            good[lbls == lbl] = True

    filtered_full[y0:y1, x0:x1] = good
    return filtered_full


def red_vs_green_score(red_mask, green_mask, blur_ksize=51):
    k = (blur_ksize, blur_ksize)
    r = cv2.blur(red_mask.astype(np.float32), k)
    g = cv2.blur(green_mask.astype(np.float32), k)
    diff = r - g
    amax = float(np.max(np.abs(diff))) + 1e-6
    norm = (diff / (2*amax) + 0.5)
    return np.clip(norm * 255, 0, 255).astype(np.uint8)


def draw_triangle(img, x, y, size=TRI_SIZE_PX, colour=PURPLE):
    h = int(size * 1.2)
    pts = np.array([[x, y], [x-size, y+h], [x+size, y+h]], np.int32)
    cv2.fillConvexPoly(img, pts, colour)
    cv2.polylines(img, [pts.reshape(-1,1,2)], True, (0,0,0), 1, cv2.LINE_AA)


def assemble_canvas(left, right, heat):
    top    = np.hstack((left, right))
    canvas = np.vstack((top, heat))
    return canvas

def maybe_downscale(img, max_w=MAX_DISPLAY_W):
    h, w = img.shape[:2]
    if w <= max_w:
        return img
    scale = max_w / float(w)
    new_size = (int(w*scale), int(h*scale))
    return cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)

# ------------------------------------------------------------------
def process_frame(img_bgr):
    """
    One-stop processing:
    • RF-DETR (Jake bbox)
    • YOLO-seg once
    • build left/right/heat
    • tint object under Jake pink on the LEFT pane, draw Jake bbox
    """
    H, W = img_bgr.shape[:2]

    # RF-DETR: Jake bbox (use PIL once; reuse the same pixels)
    pil_img = Image.fromarray(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
    try:
        dets = jake_model.predict(pil_img, threshold=0.5, device=det_device)[0]
    except NotImplementedError:
        dets = jake_model.predict(pil_img, threshold=0.5, device="cpu")[0]

    jake_xyxy = None
    if hasattr(dets, "xyxy") and len(dets.xyxy) > 0:
        x1, y1, x2, y2 = dets.xyxy[0].astype(int).tolist()
        # clamp to image
        x1 = max(0, min(W-1, x1)); x2 = max(0, min(W-1, x2))
        y1 = max(0, min(H-1, y1)); y2 = max(0, min(H-1, y2))
        if x2 > x1 and y2 > y1:
            jake_xyxy = (x1, y1, x2, y2)

    # YOLO segmentation ONCE (use original BGR frame)
    res = yolo_model.predict(img_bgr, task="segment", imgsz=IMG_SIZE,
                             device=yolo_device, conf=CONF, iou=IOU,
                             max_det=30, verbose=False, half=half)[0]

    if res.masks is None:
        # nothing detected; return pass-through panes
        left = img_bgr.copy()
        right = img_bgr.copy()
        heat_col = cv2.applyColorMap(np.zeros((H, W), np.uint8), cv2.COLORMAP_JET)
        # draw Jake bbox even if no masks
        if jake_xyxy:
            x1, y1, x2, y2 = jake_xyxy
            cv2.rectangle(left, (x1, y1), (x2, y2), JAKE_BOX_CLR, 2)
        return left, right, heat_col

    masks   = res.masks.data.cpu().numpy()  # (N, h_m, w_m)
    classes = res.masks.cls.cpu().numpy() if hasattr(res.masks, "cls") \
              else res.boxes.cls.cpu().numpy()
    classes = classes.astype(int)
    h_m, w_m = masks.shape[1:]

    # Build rail mask union at low-res then resize once
    rail_union = np.zeros((h_m, w_m), dtype=bool)
    for m, c in zip(masks, classes):
        if int(c) == RAIL_ID:
            rail_union |= m.astype(bool)
    rail_mask = cv2.resize(rail_union.astype(np.uint8), (W, H),
                           interpolation=cv2.INTER_NEAREST).astype(bool)

    # Green/red rail segmentation
    green = highlight_rails_mask_only_fast(img_bgr, rail_mask,
                                           TARGET_COLORS_RGB, TOLERANCE,
                                           MIN_REGION_SIZE, MIN_REGION_HEIGHT)
    red   = rail_mask & ~green
    score = red_vs_green_score(red, green, HEAT_BLUR_KSIZE)

    # Exclude top/bottom bands
    top_ex = int(H * EXCLUDE_TOP_FRAC)
    bot_ex = int(H * EXCLUDE_BOTTOM_FRAC)
    score[:top_ex, :] = 0
    if bot_ex: score[H-bot_ex:, :] = 0

    dark = (score >= RED_SCORE_THRESH).astype(np.uint8)
    dark = cv2.morphologyEx(
        dark, cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_RECT, (5, 9)),
        iterations=1
    )

    # Total dark area + 15% fraction threshold (tiny overhead)
    total_dark_area   = int(dark.sum())
    frac_area_thresh  = int(np.ceil(MIN_DARK_FRACTION * total_dark_area))

    # LEFT PANE: labels overlaid + Jake tinting of object above him
    left    = img_bgr.copy()
    overlay = left.copy()

    # First pass: overlay class masks (non-rail) with fixed colours + labels
    # We'll also compute which mask Jake is under with low-res overlap (no per-mask resize).
    best_idx, best_area, best_cid = None, 0, None
    if jake_xyxy:
        x1, y1, x2, y2 = jake_xyxy
        sx, sy = w_m / W, h_m / H
        mx1, mx2 = max(0, int(x1 * sx)), min(w_m, int(x2 * sx))
        my1, my2 = max(0, int(y1 * sy)), min(h_m, int(y2 * sy))
    else:
        mx1 = mx2 = my1 = my2 = None

    for idx, cid in enumerate(classes):
        if cid != RAIL_ID:
            m_low = masks[idx]  # low-res
            # Draw class overlay (need full-res mask just for drawing)
            mask_full = cv2.resize(m_low.astype(np.uint8), (W, H),
                                   interpolation=cv2.INTER_NEAREST).astype(bool)
            colour = CLASS_COLOURS.get(int(cid), (255, 255, 255))
            overlay[mask_full] = colour

            # label position
            ys, xs = np.where(mask_full)
            if len(xs):
                x_c, y_c = int(xs.mean()), int(ys.mean())
                label = obstacle_classes.get(int(cid), f"CLASS {int(cid)}")
                cv2.putText(overlay, label, (x_c-40, y_c),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 2, cv2.LINE_AA)
                cv2.putText(overlay, label, (x_c-40, y_c),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)

            # If Jake exists, compute overlap at low-res without resizing every mask
            if jake_xyxy and (mx2 > mx1) and (my2 > my1):
                area = int(m_low[my1:my2, mx1:mx2].sum())
                if area > best_area:
                    best_idx, best_area, best_cid = idx, area, int(cid)

    left = cv2.addWeighted(overlay, 0.6, left, 0.4, 0)

    # Purple-triangle warnings on LEFT (apply area thresholds, including 15%)
    n_lbl, lbl_mat, stats, _ = cv2.connectedComponentsWithStats(dark, 8)
    for lbl in range(1, n_lbl):
        area = stats[lbl, cv2.CC_STAT_AREA]
        if area < MIN_DARK_RED_AREA or area < frac_area_thresh:
            continue
        ys, xs = np.where(lbl_mat == lbl)
        y_top  = ys.min()
        x_mid  = int(xs[ys == ys.min()].mean())
        draw_triangle(left, x_mid, y_top)

    # If Jake was detected, draw his bbox and tint the object he's under PINK on LEFT
    if jake_xyxy:
        x1, y1, x2, y2 = jake_xyxy
        cv2.rectangle(left, (x1, y1), (x2, y2), JAKE_BOX_CLR, 2)
        if best_idx is not None:
            # Resize only the best mask once to full-res for the tint
            best_mask_full = cv2.resize(masks[best_idx].astype(np.uint8), (W, H),
                                        interpolation=cv2.INTER_NEAREST).astype(bool)
            pink_layer = left.copy()
            pink_layer[best_mask_full] = UNDER_TINT_BGR
            left = cv2.addWeighted(pink_layer, 0.35, left, 0.65, 0)

    # RIGHT PANE: rails tinted (red) and green highlights
    right = img_bgr.copy()
    rails_tinted = right.copy()
    rails_tinted[rail_mask] = (0,0,255)          # red tint
    right = cv2.addWeighted(rails_tinted, ALPHA, right, 1-ALPHA, 0)
    right[green] = (0,255,0)                     # green for colour-matched

    # BOTTOM: heat-map
    heat_col = cv2.applyColorMap(score, cv2.COLORMAP_JET)
    heat_col = cv2.resize(heat_col, (left.shape[1] + right.shape[1], H),
                          interpolation=cv2.INTER_LINEAR)

    return left, right, heat_col

# ------------------------------------------------------------------
if __name__ == "__main__":
    folder = Path.home() / "Documents" / "GitHub" / "Ai-plays-SubwaySurfers" / "frames"
    if not folder.is_dir():
        print(f"frames folder not found: {folder}", file=sys.stderr); sys.exit(1)

    paths = sorted(
        glob.glob(str(folder / "frame_*.jpg")) +
        glob.glob(str(folder / "frame_*.png")) +
        glob.glob(str(folder / "*.jpg")) +
        glob.glob(str(folder / "*.png"))
    )
    if not paths:
        print(f"No frame images found in {folder}", file=sys.stderr); sys.exit(1)

    cv2.namedWindow(WIN_NAME, cv2.WINDOW_NORMAL)

    i = 0
    n = len(paths)
    while i < n:
        p = paths[i]
        frame = cv2.imread(p)
        if frame is None:
            print(f"[WARN] unreadable image: {p}")
            i += 1
            continue

        try:
            t0 = time.perf_counter()
            left, right, heat = process_frame(frame)
            proc_ms = (time.perf_counter() - t0) * 1000.0
            canvas = assemble_canvas(left, right, heat)
            canvas = maybe_downscale(canvas, MAX_DISPLAY_W)

            # On-screen text for processing time
            cv2.putText(canvas, f"{os.path.basename(p)}  |  proc: {proc_ms:.1f} ms",
                        (12, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 3, cv2.LINE_AA)
            cv2.putText(canvas, f"{os.path.basename(p)}  |  proc: {proc_ms:.1f} ms",
                        (12, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.8, TEXT_CLR, 1, cv2.LINE_AA)
            cv2.putText(canvas, "SPACE: next   q/ESC: quit",
                        (12, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 3, cv2.LINE_AA)
            cv2.putText(canvas, "SPACE: next   q/ESC: quit",
                        (12, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, TEXT_CLR, 1, cv2.LINE_AA)

        except Exception as e:
            print(f"[WARN] error on {os.path.basename(p)}: {e}")
            i += 1
            continue

        # Responsive event loop: SPACE to advance, q/ESC to quit
        while True:
            cv2.imshow(WIN_NAME, canvas)
            key = cv2.waitKey(1) & 0xFF

            if cv2.getWindowProperty(WIN_NAME, cv2.WND_PROP_VISIBLE) < 1:
                cv2.destroyAllWindows(); sys.exit(0)

            if key == 32:          # SPACE → next
                i += 1
                break
            elif key in (ord('q'), 27):  # q or ESC
                cv2.destroyAllWindows(); sys.exit(0)

    cv2.destroyAllWindows()


In [ ]:
#!/usr/bin/env python3
"""
Speed-test the integrated RF-DETR (Jake bbox) + YOLO-seg pipeline.

• Uses the same processing logic as the interactive viewer, but with NO GUI.
• Runs through all images in: ~/Documents/GitHub/Ai-plays-SubwaySurfers/frames
• First 10 frames are executed as WARMUP (not timed).
• Prints per-frame processing time (after warmup) and a summary at the end.
"""

import os, sys, glob, time, statistics
# ── Enable CPU fallback before importing torch (for MPS gaps) ────────────────
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import cv2
import numpy as np
import torch
import torch.nn.functional as F
from pathlib import Path
from PIL import Image
from ultralytics import YOLO
from rfdetr import RFDETRBase

# ── Monkey-patch interpolate to disable antialias (avoids unsupported MPS op) ─
_original_interpolate = F.interpolate
def _interpolate_no_antialias(input, *args, **kwargs):
    kwargs['antialias'] = False
    return _original_interpolate(input, *args, **kwargs)
F.interpolate = _interpolate_no_antialias

# ─────────────────────────────────────────────────────────── Config ──
home     = os.path.expanduser("~")
yolo_weights = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
jake_weights = f"{home}/downloads/weightsjake.pt"

RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40

TARGET_COLORS_RGB  = [(119,104,67), (81,42,45)]
TOLERANCE          = 20.0
MIN_REGION_SIZE    = 30
MIN_REGION_HEIGHT  = 150

HEAT_BLUR_KSIZE     = 51
RED_SCORE_THRESH    = 220
EXCLUDE_TOP_FRAC    = 0.40
EXCLUDE_BOTTOM_FRAC = 0.15
MIN_DARK_RED_AREA   = 1200
TRI_SIZE_PX         = 18
PURPLE              = (255, 0, 255)

# Purple-triangle extra filter: blob must be ≥ this fraction of total dark area
MIN_DARK_FRACTION   = 0.15  # 15%

WARMUP_FRAMES       = 10

# ─────────────────────────────────────────────── Device / model init ─
# YOLO device: CUDA > MPS > CPU
if torch.cuda.is_available():
    yolo_device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    yolo_device, half = "mps", False
else:
    yolo_device, half = "cpu", False

# RF-DETR device: prefer MPS if present, else CPU
det_device = "mps" if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available() else "cpu"

# Optional: reduce OpenCV CPU thread contention with PyTorch
try:
    cv2.setNumThreads(1)
except Exception:
    pass

yolo_model = YOLO(yolo_weights)
try: yolo_model.fuse()
except Exception: pass

jake_model = RFDETRBase(pretrain_weights=jake_weights, num_classes=3)

# Warm-up YOLO so first frame isn't laggy
_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = yolo_model.predict(_dummy, task="segment", imgsz=IMG_SIZE,
                       device=yolo_device, conf=CONF, iou=IOU,
                       verbose=False, half=half)

# ─────────────────────────────────────── Helper processing functions ─
def highlight_rails_mask_only_fast(img_bgr, rail_mask, target_colors_rgb,
                                   tolerance=30.0,
                                   min_region_size=50,
                                   min_region_height=150):
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max()+1
    x0, x1 = xs.min(), xs.max()+1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    targets_bgr = np.array([(r,g,b)[::-1] for r,g,b in target_colors_rgb],
                           dtype=np.float32)
    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - targets_bgr[None, None, :, :]
    dist2 = np.sum(diff*diff, axis=-1)
    colour_hit = np.any(dist2 <= tolerance**2, axis=-1)

    combined = colour_hit & mask_roi
    comp = combined.astype(np.uint8)
    n, lbls, stats, _ = cv2.connectedComponentsWithStats(comp, 8)

    good = np.zeros_like(combined)
    for lbl in range(1, n):
        area, h = stats[lbl, cv2.CC_STAT_AREA], stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and h >= min_region_height:
            good[lbls == lbl] = True

    filtered_full[y0:y1, x0:x1] = good
    return filtered_full


def red_vs_green_score(red_mask, green_mask, blur_ksize=51):
    k = (blur_ksize, blur_ksize)
    r = cv2.blur(red_mask.astype(np.float32), k)
    g = cv2.blur(green_mask.astype(np.float32), k)
    diff = r - g
    amax = float(np.max(np.abs(diff))) + 1e-6
    norm = (diff / (2*amax) + 0.5)
    return np.clip(norm * 255, 0, 255).astype(np.uint8)


def draw_triangle(img, x, y, size=TRI_SIZE_PX, colour=PURPLE):
    h = int(size * 1.2)
    pts = np.array([[x, y], [x-size, y+h], [x+size, y+h]], np.int32)
    cv2.fillConvexPoly(img, pts, colour)
    cv2.polylines(img, [pts.reshape(-1,1,2)], True, (0,0,0), 1, cv2.LINE_AA)

# ─────────────────────────────────────────── Pipeline per frame ─
def process_frame(img_bgr):
    """
    Core pipeline used for timing:
    • RF-DETR (Jake bbox) once
    • YOLO-seg once
    • rail green/red + heat computation
    • purple triangle logic with 15% total dark-area filter
    We do NOT build/display the 3-panel canvas here (no GUI).
    """
    H, W = img_bgr.shape[:2]

    # RF-DETR: Jake bbox (PIL once)
    pil_img = Image.fromarray(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
    try:
        dets = jake_model.predict(pil_img, threshold=0.5, device=det_device)[0]
    except NotImplementedError:
        dets = jake_model.predict(pil_img, threshold=0.5, device="cpu")[0]

    # YOLO segmentation ONCE (use original BGR frame)
    res = yolo_model.predict(img_bgr, task="segment", imgsz=IMG_SIZE,
                             device=yolo_device, conf=CONF, iou=IOU,
                             max_det=30, verbose=False, half=half)[0]

    if res.masks is None:
        return  # nothing detected; timing still counts

    masks   = res.masks.data.cpu().numpy()  # (N, h_m, w_m)
    classes = res.masks.cls.cpu().numpy() if hasattr(res.masks, "cls") \
              else res.boxes.cls.cpu().numpy()
    classes = classes.astype(int)
    h_m, w_m = masks.shape[1:]

    # Rail union
    rail_union = np.zeros((h_m, w_m), dtype=bool)
    for m, c in zip(masks, classes):
        if int(c) == RAIL_ID:
            rail_union |= m.astype(bool)
    rail_mask = cv2.resize(rail_union.astype(np.uint8), (W, H),
                           interpolation=cv2.INTER_NEAREST).astype(bool)

    # Green/red rails + heat
    green = highlight_rails_mask_only_fast(img_bgr, rail_mask,
                                           TARGET_COLORS_RGB, TOLERANCE,
                                           MIN_REGION_SIZE, MIN_REGION_HEIGHT)
    red   = rail_mask & ~green
    score = red_vs_green_score(red, green, HEAT_BLUR_KSIZE)

    # Exclude top/bottom bands
    top_ex = int(H * EXCLUDE_TOP_FRAC)
    bot_ex = int(H * EXCLUDE_BOTTOM_FRAC)
    score[:top_ex, :] = 0
    if bot_ex: score[H-bot_ex:, :] = 0

    dark = (score >= RED_SCORE_THRESH).astype(np.uint8)
    dark = cv2.morphologyEx(
        dark, cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_RECT, (5, 9)),
        iterations=1
    )

    # purple triangles (area checks incl. 15% of total dark area)
    total_dark_area  = int(dark.sum())
    frac_area_thresh = int(np.ceil(MIN_DARK_FRACTION * total_dark_area))
    n_lbl, lbl_mat, stats, _ = cv2.connectedComponentsWithStats(dark, 8)
    # (we don't draw here – timing only; this loop is kept for parity with viewer)
    for lbl in range(1, n_lbl):
        area = stats[lbl, cv2.CC_STAT_AREA]
        if area < MIN_DARK_RED_AREA or area < frac_area_thresh:
            continue
        # would draw triangle in viewer version

# ──────────────────────────────────────────────────────── Main test ──
if __name__ == "__main__":
    folder = (Path.home() / "Documents" / "GitHub" /
              "Ai-plays-SubwaySurfers" / "frames")

    if not folder.is_dir():
        print(f"[ERROR] frames folder not found: {folder}", file=sys.stderr)
        sys.exit(1)

    paths = sorted(
        glob.glob(str(folder / "frame_*.jpg")) +
        glob.glob(str(folder / "frame_*.png")) +
        glob.glob(str(folder / "*.jpg")) +
        glob.glob(str(folder / "*.png"))
    )
    if not paths:
        print(f"[ERROR] No frame images found in {folder}", file=sys.stderr)
        sys.exit(1)

    times = []
    t_start = time.perf_counter()

    # Run with warmup
    for idx, p in enumerate(paths):
        frame = cv2.imread(p)
        if frame is None:
            print(f"[WARN] unreadable image: {p}")
            continue

        if idx < WARMUP_FRAMES:
            # Warm-up: execute but don't time or print
            process_frame(frame)
            continue

        t0 = time.perf_counter()
        process_frame(frame)
        dt = time.perf_counter() - t0
        times.append(dt)

        # Per-frame timing print (post-warmup only)
        print(f"{os.path.basename(p):>20s}  {dt*1000:7.2f} ms")

    total_time = time.perf_counter() - t_start
    timed_frames = len(times)

    # ─────────────────────────────── Summary ─
    if timed_frames == 0:
        print("\n[WARNING] fewer frames than WARMUP_FRAMES; nothing timed.")
        sys.exit(0)

    ms = [t * 1_000 for t in times]
    avg = statistics.mean(ms)
    med = statistics.median(ms)
    print("\n───── Speed-test summary (warm-up skipped) ─────")
    print(f"Total frames           : {len(paths)}")
    print(f"Warm-up frames ignored : {WARMUP_FRAMES}")
    print(f"Frames timed           : {timed_frames}")
    print(f"Total wall-clock time  : {total_time:,.2f} s")
    print(f"Average / frame        : {avg:,.2f} ms  ({1_000/avg:,.2f} FPS)")
    print(f"Median                 : {med:,.2f} ms")
    print(f"Fastest                : {min(ms):,.2f} ms")
    print(f"Slowest                : {max(ms):,.2f} ms")
    print("───────────────────────────────────────────────")


In [ ]:
#More gassss

In [ ]:
#!/usr/bin/env python3
"""
Speed-test the integrated RF-DETR (Jake bbox) + YOLO-seg pipeline.

• Same processing logic as the interactive viewer, but with NO GUI.
• Scans:  ~/Documents/GitHub/Ai-plays-SubwaySurfers/frames
• First 10 frames are WARMUP (not timed).
• Prints per-frame time (after warmup) and a summary at the end.
"""

import os, sys, glob, time, statistics
# ── Enable CPU fallback before importing torch (for MPS gaps) ────────────────
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import cv2
import numpy as np
import torch
import torch.nn.functional as F
from pathlib import Path
from PIL import Image
from ultralytics import YOLO
from rfdetr import RFDETRBase

# ── Monkey-patch interpolate to disable antialias (avoids unsupported MPS op) ─
_original_interpolate = F.interpolate
def _interpolate_no_antialias(input, *args, **kwargs):
    kwargs['antialias'] = False
    return _original_interpolate(input, *args, **kwargs)
F.interpolate = _interpolate_no_antialias

# ─────────────────────────────────────────────────────────── Config ──
home     = os.path.expanduser("~")
yolo_weights = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
jake_weights = f"{home}/downloads/weightsjake.pt"

RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40  # (kept for parity; no GUI draw here)

TARGET_COLORS_RGB  = [(119,104,67), (81,42,45)]
TOLERANCE          = 20.0
MIN_REGION_SIZE    = 30
MIN_REGION_HEIGHT  = 150

HEAT_BLUR_KSIZE     = 51
RED_SCORE_THRESH    = 220
EXCLUDE_TOP_FRAC    = 0.40
EXCLUDE_BOTTOM_FRAC = 0.15
MIN_DARK_RED_AREA   = 1200
TRI_SIZE_PX         = 18
PURPLE              = (255, 0, 255)

# Purple-triangle extra filter: blob must be ≥ this fraction of total dark area
MIN_DARK_FRACTION   = 0.15  # 15%

WARMUP_FRAMES       = 10

# ─────────────────────────────────────────────── Device / model init ─
# YOLO device: CUDA > MPS > CPU
if torch.cuda.is_available():
    yolo_device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    yolo_device, half = "mps", False
else:
    yolo_device, half = "cpu", False

# RF-DETR device: prefer MPS if present, else CPU
det_device = "mps" if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available() else "cpu"

# Optional: reduce OpenCV CPU thread contention with PyTorch
try:
    cv2.setNumThreads(1)
except Exception:
    pass
cv2.setUseOptimized(True)

# ── Load models once ────────────────────────────────────────────────────────
yolo_model = YOLO(yolo_weights)
try:
    yolo_model.fuse()
except Exception:
    pass

jake_model = RFDETRBase(pretrain_weights=jake_weights, num_classes=3)

# ── Precompute constants used every frame ───────────────────────────────────
# target colors -> BGR float32, tolerance^2
_TARGETS_BGR_F32 = np.array([(r, g, b)[::-1] for (r, g, b) in TARGET_COLORS_RGB], dtype=np.float32)
_TOL2 = float(TOLERANCE * TOLERANCE)
# morphology kernel reused
_MORPH_KERNEL = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 9))

# ── Warm-up both models so first timed frame isn't polluted ─────────────────
_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
with torch.inference_mode():
    _ = yolo_model.predict(
        _dummy, task="segment", imgsz=IMG_SIZE,
        device=yolo_device, conf=CONF, iou=IOU,
        verbose=False, half=half
    )
# RF-DETR expects PIL; use tiny gray image to compile weights/graphs
try:
    _pil_dummy = Image.fromarray(np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8))
    _ = jake_model.predict(_pil_dummy, threshold=0.5, device=det_device)
except NotImplementedError:
    _ = jake_model.predict(_pil_dummy, threshold=0.5, device="cpu")

# ─────────────────────────────────────── Helper processing functions ─
def highlight_rails_mask_only_fast(img_bgr, rail_mask):
    """
    Color + size filter inside rail_mask.
    Uses precomputed _TARGETS_BGR_F32 and _TOL2, avoids Python loops.
    """
    # Quick reject
    if rail_mask is None or not rail_mask.any():
        return np.zeros(rail_mask.shape, dtype=bool)

    H, W = img_bgr.shape[:2]
    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    # Compute min color distance to any target in a vectorized way
    img_f = img_roi.astype(np.float32, copy=False)
    diff  = img_f[:, :, None, :] - _TARGETS_BGR_F32[None, None, :, :]
    dist2 = np.einsum("...c,...c->...", diff, diff)  # sum over BGR
    # dist2 shape: (h, w, n_targets) -> min across targets
    if dist2.ndim == 3:
        min_dist2 = dist2.min(axis=2)
    else:
        min_dist2 = dist2  # just in case

    colour_hit = (min_dist2 <= _TOL2)
    combined = colour_hit & mask_roi

    if not combined.any():
        out = np.zeros((H, W), dtype=bool)
        return out

    comp = combined.view(np.uint8)
    n, lbls, stats, _ = cv2.connectedComponentsWithStats(comp, 8)

    if n <= 1:
        out = np.zeros((H, W), dtype=bool)
        return out

    good = np.zeros_like(comp, dtype=np.uint8)
    # Vectorized filter for components (skip label 0)
    areas  = stats[1:, cv2.CC_STAT_AREA]
    hs     = stats[1:, cv2.CC_STAT_HEIGHT]
    keep   = (areas >= MIN_REGION_SIZE) & (hs >= MIN_REGION_HEIGHT)
    if not keep.any():
        out = np.zeros((H, W), dtype=bool)
        return out

    # Paint kept labels
    kept_labels = (np.where(keep)[0] + 1).tolist()
    for lbl in kept_labels:
        good[lbls == lbl] = 1

    out = np.zeros((H, W), dtype=bool)
    out[y0:y1, x0:x1] = good.astype(bool)
    return out


def red_vs_green_score(red_mask, green_mask):
    # cv2.blur is an optimized box filter; keep it for large kernels
    k = (HEAT_BLUR_KSIZE, HEAT_BLUR_KSIZE)
    r = cv2.blur(red_mask.astype(np.float32, copy=False), k)
    g = cv2.blur(green_mask.astype(np.float32, copy=False), k)
    diff = r - g
    # Normalize to 0..255 without repeated max calls
    amax = float(np.max(diff)) if np.max(diff) > -np.min(diff) else float(-np.min(diff))
    amax = amax + 1e-6
    norm = (diff / (2 * amax) + 0.5)
    return np.clip(norm * 255.0, 0.0, 255.0).astype(np.uint8)


@torch.inference_mode()
def yolo_seg_once(img_bgr):
    return yolo_model.predict(
        img_bgr, task="segment", imgsz=IMG_SIZE,
        device=yolo_device, conf=CONF, iou=IOU,
        max_det=30, verbose=False, half=half
    )[0]


def process_frame(img_bgr):
    """
    Core pipeline used for timing:
    • RF-DETR (Jake bbox) once (kept for parity)
    • YOLO-seg once
    • rail green/red + heat computation
    • purple triangle logic with 15% total dark-area filter
    No GUI work here.
    """
    H, W = img_bgr.shape[:2]

    # RF-DETR: Jake bbox (PIL). We call it to mirror the viewer cost,
    # but don't consume results here.
    pil_img = Image.fromarray(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
    try:
        _ = jake_model.predict(pil_img, threshold=0.5, device=det_device)[0]
    except NotImplementedError:
        _ = jake_model.predict(pil_img, threshold=0.5, device="cpu")[0]

    # YOLO segmentation ONCE
    res = yolo_seg_once(img_bgr)
    masks_obj = res.masks
    if masks_obj is None:
        return  # nothing detected; timing still counts

    # Prefer boxes.cls (always present); fall back if needed
    try:
        classes = res.boxes.cls.detach().cpu().numpy().astype(int)
    except Exception:
        classes = (masks_obj.cls.detach().cpu().numpy().astype(int)
                   if hasattr(masks_obj, "cls") else None)

    masks = masks_obj.data.detach().cpu().numpy().astype(bool)  # (N, h_m, w_m)
    if classes is None or masks.size == 0:
        return

    # Rail union without Python loop
    rail_idx = (classes == RAIL_ID)
    if not rail_idx.any():
        return
    rail_union_small = np.any(masks[rail_idx, :, :], axis=0)
    rail_mask = cv2.resize(rail_union_small.astype(np.uint8), (W, H),
                           interpolation=cv2.INTER_NEAREST).astype(bool)

    if not rail_mask.any():
        return

    # Green/red rails + heat
    green = highlight_rails_mask_only_fast(img_bgr, rail_mask)
    red   = rail_mask & ~green

    if not red.any() and not green.any():
        return

    score = red_vs_green_score(red, green)

    # Exclude top/bottom bands (index math only, no copies)
    top_ex = int(H * EXCLUDE_TOP_FRAC)
    bot_ex = int(H * EXCLUDE_BOTTOM_FRAC)
    if top_ex > 0:
        score[:top_ex, :] = 0
    if bot_ex > 0:
        score[H - bot_ex:, :] = 0

    # Threshold to "dark" and do a single morphology open
    # (branchless threshold via cv2 works fine; >= RED_SCORE_THRESH)
    _, dark = cv2.threshold(score, RED_SCORE_THRESH - 1, 255, cv2.THRESH_BINARY)
    if not np.any(dark):
        return
    dark = cv2.morphologyEx(dark, cv2.MORPH_OPEN, _MORPH_KERNEL, iterations=1)

    # purple triangles (area checks incl. 15% of total dark area)
    total_dark_area = int(cv2.countNonZero(dark))
    if total_dark_area == 0:
        return

    frac_area_thresh = int(np.ceil(MIN_DARK_FRACTION * total_dark_area))
    n_lbl, lbl_mat, stats, _ = cv2.connectedComponentsWithStats(dark, 8)
    if n_lbl <= 1:
        return

    # Filter blobs via vector ops; we don't draw in this headless speed test
    areas = stats[1:, cv2.CC_STAT_AREA]
    keep  = (areas >= MIN_DARK_RED_AREA) & (areas >= frac_area_thresh)
    # If needed, we could short-circuit on keep.any(), but either way we’re done.


# ──────────────────────────────────────────────────────── Main test ──
if __name__ == "__main__":
    folder = (Path.home() / "Documents" / "GitHub" /
              "Ai-plays-SubwaySurfers" / "frames")

    if not folder.is_dir():
        print(f"[ERROR] frames folder not found: {folder}", file=sys.stderr)
        sys.exit(1)

    # Prefer the "frame_*.{jpg,png}" pattern first; fallback to all images
    paths = sorted(
        glob.glob(str(folder / "frame_*.jpg"))
        + glob.glob(str(folder / "frame_*.png"))
        + glob.glob(str(folder / "*.jpg"))
        + glob.glob(str(folder / "*.png"))
    )
    if not paths:
        print(f"[ERROR] No frame images found in {folder}", file=sys.stderr)
        sys.exit(1)

    times = []
    t_start = time.perf_counter()

    # Run with warmup
    for idx, p in enumerate(paths):
        frame = cv2.imread(p, cv2.IMREAD_COLOR)
        if frame is None:
            print(f"[WARN] unreadable image: {p}")
            continue

        if idx < WARMUP_FRAMES:
            process_frame(frame)
            continue

        t0 = time.perf_counter()
        process_frame(frame)
        dt = time.perf_counter() - t0
        times.append(dt)

        # Per-frame timing print (post-warmup only)
        print(f"{os.path.basename(p):>20s}  {dt*1000:7.2f} ms")

    total_time = time.perf_counter() - t_start
    timed_frames = len(times)

    # ─────────────────────────────── Summary ─
    if timed_frames == 0:
        print("\n[WARNING] fewer frames than WARMUP_FRAMES; nothing timed.")
        sys.exit(0)

    ms = [t * 1_000 for t in times]
    avg = statistics.mean(ms)
    med = statistics.median(ms)
    print("\n───── Speed-test summary (warm-up skipped) ─────")
    print(f"Total frames           : {len(paths)}")
    print(f"Warm-up frames ignored : {WARMUP_FRAMES}")
    print(f"Frames timed           : {timed_frames}")
    print(f"Total wall-clock time  : {total_time:,.2f} s")
    print(f"Average / frame        : {avg:,.2f} ms  ({1_000/avg:,.2f} FPS)")
    print(f"Median                 : {med:,.2f} ms")
    print(f"Fastest                : {min(ms):,.2f} ms")
    print(f"Slowest                : {max(ms):,.2f} ms")
    print("───────────────────────────────────────────────")


In [ ]:
#Try to spread load of CPU and GPU

In [ ]:
#!/usr/bin/env python3
"""
Speed-test: run the full segmentation/overlay pipeline on every image in
~/Documents/GitHub/Ai-plays-SubwaySurfers/frames and print a timing
summary at the end.

Changes in this version
────────────────────────
• The first WARMUP_FRAMES (default 10) are *executed* but **not timed**,
  so model/kernel warm-up and disk cache effects don’t skew results.
• Everything else is unchanged: no GUI overhead, concise summary.
"""

import os, sys, glob, time, statistics
from pathlib import Path

import cv2
import numpy as np
import torch
from ultralytics import YOLO

# ───────────────────────────────────────────────────────── Config ──
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40

TARGET_COLORS_RGB  = [(119,104,67), (81,42,45)]
TOLERANCE          = 20.0
MIN_REGION_SIZE    = 30
MIN_REGION_HEIGHT  = 150

HEAT_BLUR_KSIZE     = 51
RED_SCORE_THRESH    = 220
EXCLUDE_TOP_FRAC    = 0.40
EXCLUDE_BOTTOM_FRAC = 0.15
MIN_DARK_RED_AREA   = 1200
TRI_SIZE_PX         = 18
PURPLE              = (255, 0, 255)

WARMUP_FRAMES       = 10   # how many frames to skip from timing stats

obstacle_classes = {
    0: "BOOTS", 1: "GREYTRAIN", 2: "HIGHBARRIER1", 3: "JUMP", 4: "LOWBARRIER1",
    5: "LOWBARRIER2", 6: "ORANGETRAIN", 7: "PILLAR", 8: "RAMP", 9: "RAILS",
    10: "SIDEWALK", 11: "YELLOWTRAIN"
}
CLASS_COLOURS = {
    0: (255,255,0), 1: (192,192,192), 2: (0,128,255), 3: (0,255,0),
    4: (255,0,255), 5: (0,255,255), 6: (255,128,0), 7: (128,0,255),
    8: (0,0,128), 10: (128,128,0), 11: (255,255,102)
}

# ─────────────────────────────────────── Device / model init ─
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

model = YOLO(weights)
try: model.fuse()
except Exception: pass
model.predict(
    np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8),
    task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, verbose=False, half=half
)

# ───────────────────────────── Helper processing functions ─
def highlight_rails_mask_only_fast(img_bgr, rail_mask, target_colors_rgb,
                                   tolerance=30.0,
                                   min_region_size=50,
                                   min_region_height=150):
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max()+1
    x0, x1 = xs.min(), xs.max()+1
    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    targets_bgr = np.array([(r,g,b)[::-1] for r,g,b in target_colors_rgb],
                           dtype=np.float32)
    diff  = img_roi.astype(np.float32)[:, :, None, :] - targets_bgr[None, None]
    dist2 = np.sum(diff*diff, axis=-1)
    colour_hit = np.any(dist2 <= tolerance**2, axis=-1)

    combined = colour_hit & mask_roi
    comp = combined.astype(np.uint8)
    n, lbls, stats, _ = cv2.connectedComponentsWithStats(comp, 8)

    good = np.zeros_like(combined)
    for lbl in range(1, n):
        area, h = stats[lbl, cv2.CC_STAT_AREA], stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and h >= min_region_height:
            good[lbls == lbl] = True

    filtered_full[y0:y1, x0:x1] = good
    return filtered_full


def red_vs_green_score(red_mask, green_mask, blur_ksize=51):
    k = (blur_ksize, blur_ksize)
    diff = (cv2.blur(red_mask.astype(np.float32), k) -
            cv2.blur(green_mask.astype(np.float32), k))
    norm = (diff / (2*(np.max(np.abs(diff))+1e-6)) + 0.5)
    return np.clip(norm * 255, 0, 255).astype(np.uint8)


def process_frame(img_bgr):
    res = model.predict(img_bgr, task="segment", imgsz=IMG_SIZE,
                        device=device, conf=CONF, iou=IOU,
                        max_det=30, verbose=False, half=half)[0]
    if res.masks is None:
        return

    masks   = res.masks.data.cpu().numpy()
    classes = res.masks.cls.cpu().numpy() if hasattr(res.masks, "cls") \
              else res.boxes.cls.cpu().numpy()

    H, W = img_bgr.shape[:2]
    rail_union = np.zeros(masks.shape[1:], bool)
    for m, c in zip(masks, classes):
        if int(c) == RAIL_ID:
            rail_union |= m.astype(bool)

    rail_mask = cv2.resize(rail_union.astype(np.uint8), (W, H),
                           interpolation=cv2.INTER_NEAREST).astype(bool)

    green = highlight_rails_mask_only_fast(img_bgr, rail_mask,
                                           TARGET_COLORS_RGB, TOLERANCE,
                                           MIN_REGION_SIZE, MIN_REGION_HEIGHT)
    red   = rail_mask & ~green
    score = red_vs_green_score(red, green, HEAT_BLUR_KSIZE)

    # exclude top/bottom
    score[:int(H*EXCLUDE_TOP_FRAC), :] = 0
    score[H-int(H*EXCLUDE_BOTTOM_FRAC):, :] = 0

    dark = (score >= RED_SCORE_THRESH).astype(np.uint8)
    cv2.morphologyEx(
        dark, cv2.MORPH_OPEN,
        cv2.getStructuringElement(cv2.MORPH_RECT, (5, 9)),
        dst=dark, iterations=1
    )
    # outputs are not displayed; function exists for timing only.

# ─────────────────────────────────────────────────────── Main test ──
if __name__ == "__main__":
    folder = (Path.home() / "Documents" / "GitHub" /
              "Ai-plays-SubwaySurfers" / "frames")

    if not folder.is_dir():
        print(f"[ERROR] frames folder not found: {folder}", file=sys.stderr)
        sys.exit(1)

    paths = sorted(
        glob.glob(str(folder / "frame_*.jpg")) +
        glob.glob(str(folder / "frame_*.png")) +
        glob.glob(str(folder / "*.jpg")) +
        glob.glob(str(folder / "*.png"))
    )
    if not paths:
        print(f"[ERROR] No frame images found in {folder}", file=sys.stderr)
        sys.exit(1)

    times = []
    t_start = time.perf_counter()

    for idx, p in enumerate(paths):
        frame = cv2.imread(p)

        if idx < WARMUP_FRAMES:
            # Warm-up: run but don't time
            process_frame(frame)
            continue

        t0 = time.perf_counter()
        process_frame(frame)
        times.append(time.perf_counter() - t0)
        print(time.perf_counter() - t0)

    total_time = time.perf_counter() - t_start
    timed_frames = len(times)

    # ─────────────────────────────── Summary ─
    ms = [t * 1_000 for t in times]
    if timed_frames == 0:
        print("\n[WARNING] fewer frames than WARMUP_FRAMES; nothing timed.")
        sys.exit(0)

    print("\n───── Speed-test summary (warm-up skipped) ─────")
    print(f"Total frames           : {len(paths)}")
    print(f"Warm-up frames ignored : {WARMUP_FRAMES}")
    print(f"Frames timed           : {timed_frames}")
    print(f"Total wall-clock time  : {total_time:,.2f} s")
    print(f"Average / frame        : {statistics.mean(ms):,.2f} ms"
          f"  ({1_000/statistics.mean(ms):,.2f} FPS)")
    print(f"Median                 : {statistics.median(ms):,.2f} ms")
    print(f"Fastest                : {min(ms):,.2f} ms")
    print(f"Slowest                : {max(ms):,.2f} ms")
    print("────────────────────────────────────────────────")


In [ ]:
#With prints

In [ ]:
#!/usr/bin/env python3
"""
Speed-test: run the full segmentation/overlay pipeline on every image in
~/Documents/GitHub/Ai-plays-SubwaySurfers/frames, show each processed
frame inline in Jupyter (stacked), and print a timing summary at the end.

Warm-up frames are executed but not timed or displayed.
"""

import os, sys, glob, time, statistics
from pathlib import Path

import cv2
import numpy as np
import torch
from ultralytics import YOLO

# Optional notebook display (safe fallback if not in Jupyter)
try:
    from IPython.display import display
    from PIL import Image
    _HAS_IPY = True
except Exception:
    _HAS_IPY = False

# ───────────────────────────────────────────────────────── Config ──
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40

TARGET_COLORS_RGB  = [(119,104,67), (81,42,45)]
TOLERANCE          = 20.0
MIN_REGION_SIZE    = 30
MIN_REGION_HEIGHT  = 150

HEAT_BLUR_KSIZE     = 51
RED_SCORE_THRESH    = 220
EXCLUDE_TOP_FRAC    = 0.40
EXCLUDE_BOTTOM_FRAC = 0.15
MIN_DARK_RED_AREA   = 1200
TRI_SIZE_PX         = 18
PURPLE              = (255, 0, 255)

WARMUP_FRAMES       = 10   # how many frames to skip from timing stats

# ─────────────────────────────────────── Device / model init ─
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

model = YOLO(weights)
try: model.fuse()
except Exception: pass
model.predict(
    np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8),
    task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, verbose=False, half=half
)

# ───────────────────────────── Helper processing functions ─
def highlight_rails_mask_only_fast(img_bgr, rail_mask, target_colors_rgb,
                                   tolerance=30.0,
                                   min_region_size=50,
                                   min_region_height=150):
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max()+1
    x0, x1 = xs.min(), xs.max()+1
    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    targets_bgr = np.array([(r,g,b)[::-1] for r,g,b in target_colors_rgb],
                           dtype=np.float32)
    diff  = img_roi.astype(np.float32)[:, :, None, :] - targets_bgr[None, None]
    dist2 = np.sum(diff*diff, axis=-1)
    colour_hit = np.any(dist2 <= tolerance**2, axis=-1)

    combined = colour_hit & mask_roi
    comp = combined.astype(np.uint8)
    n, lbls, stats, _ = cv2.connectedComponentsWithStats(comp, 8)

    good = np.zeros_like(combined)
    for lbl in range(1, n):
        area, h = stats[lbl, cv2.CC_STAT_AREA], stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and h >= min_region_height:
            good[lbls == lbl] = True

    filtered_full[y0:y1, x0:x1] = good
    return filtered_full


def red_vs_green_score(red_mask, green_mask, blur_ksize=51):
    k = (blur_ksize, blur_ksize)
    diff = (cv2.blur(red_mask.astype(np.float32), k) -
            cv2.blur(green_mask.astype(np.float32), k))
    norm = (diff / (2*(np.max(np.abs(diff))+1e-6)) + 0.5)
    return np.clip(norm * 255, 0, 255).astype(np.uint8)


def make_overlay_panel(img_bgr, rail_mask, green_mask, score_u8):
    """Build a side-by-side panel: original | overlays (red rail, green keep, score heat)."""
    H, W = img_bgr.shape[:2]

    # Color overlays
    overlay = img_bgr.copy()
    # red = rail minus green
    red_mask = (rail_mask & ~green_mask)

    # tint red & green
    red_layer = np.zeros_like(img_bgr);    red_layer[:,:,2] = 255
    green_layer = np.zeros_like(img_bgr);  green_layer[:,:,1] = 255

    overlay = np.where(red_mask[...,None], cv2.addWeighted(overlay, 1-ALPHA, red_layer, ALPHA, 0), overlay)
    overlay = np.where(green_mask[...,None], cv2.addWeighted(overlay, 1-ALPHA, green_layer, ALPHA, 0), overlay)

    # score to heat (JET)
    score_color = cv2.applyColorMap(score_u8, cv2.COLORMAP_JET)
    score_vis = cv2.addWeighted(img_bgr, 0.55, score_color, 0.45, 0)

    # stack: original | overlay | score
    panel = np.concatenate([img_bgr, overlay, score_vis], axis=1)

    # simple rulers/text
    cv2.putText(panel, "original", (10, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (32,32,255), 2, cv2.LINE_AA)
    cv2.putText(panel, "rail(red) / keep(green)", (W+10, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (32,255,32), 2, cv2.LINE_AA)
    cv2.putText(panel, "red-vs-green score", (2*W+10, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,32), 2, cv2.LINE_AA)

    return panel


def process_frame(img_bgr):
    """Run segmentation + masks + score; return rail_mask, green_mask, score_u8, and a display panel."""
    res = model.predict(img_bgr, task="segment", imgsz=IMG_SIZE,
                        device=device, conf=CONF, iou=IOU,
                        max_det=30, verbose=False, half=half)[0]
    if res.masks is None:
        H, W = img_bgr.shape[:2]
        empty = np.zeros((H, W), dtype=bool)
        score = np.zeros((H, W), dtype=np.uint8)
        panel = make_overlay_panel(img_bgr, empty, empty, score)
        return empty, empty, score, panel

    masks   = res.masks.data.cpu().numpy()
    classes = res.masks.cls.cpu().numpy() if hasattr(res.masks, "cls") \
              else res.boxes.cls.cpu().numpy()

    H, W = img_bgr.shape[:2]
    rail_union = np.zeros(masks.shape[1:], bool)
    for m, c in zip(masks, classes):
        if int(c) == RAIL_ID:
            rail_union |= m.astype(bool)

    rail_mask = cv2.resize(rail_union.astype(np.uint8), (W, H),
                           interpolation=cv2.INTER_NEAREST).astype(bool)

    green = highlight_rails_mask_only_fast(img_bgr, rail_mask,
                                           TARGET_COLORS_RGB, TOLERANCE,
                                           MIN_REGION_SIZE, MIN_REGION_HEIGHT)
    red   = rail_mask & ~green
    score = red_vs_green_score(red, green, HEAT_BLUR_KSIZE)

    # exclude top/bottom bands from consideration
    score[:int(H*EXCLUDE_TOP_FRAC), :] = 0
    score[H-int(H*EXCLUDE_BOTTOM_FRAC):, :] = 0

    panel = make_overlay_panel(img_bgr, rail_mask, green, score)
    return rail_mask, green, score, panel

# ─────────────────────────────────────────────────────── Main test ──
if __name__ == "__main__":
    folder = (Path.home() / "Documents" / "GitHub" /
              "Ai-plays-SubwaySurfers" / "frames")

    if not folder.is_dir():
        print(f"[ERROR] frames folder not found: {folder}", file=sys.stderr)
        sys.exit(1)

    paths = sorted(
        glob.glob(str(folder / "frame_*.jpg")) +
        glob.glob(str(folder / "frame_*.png")) +
        glob.glob(str(folder / "*.jpg")) +
        glob.glob(str(folder / "*.png"))
    )
    if not paths:
        print(f"[ERROR] No frame images found in {folder}", file=sys.stderr)
        sys.exit(1)

    times = []
    t_start = time.perf_counter()

    for idx, p in enumerate(paths):
        frame = cv2.imread(p)

        if idx < WARMUP_FRAMES:
            # Warm-up: run but don't time or display
            _ = process_frame(frame)
            continue

        t0 = time.perf_counter()
        rail_mask, green_mask, score_u8, panel_bgr = process_frame(frame)
        dt = time.perf_counter() - t0
        times.append(dt)

        # print timing line
        print(f"{idx:04d} | {Path(p).name} | {dt*1000:7.2f} ms")

        # show stacked image in Jupyter (if available)
        if _HAS_IPY:
            rgb = cv2.cvtColor(panel_bgr, cv2.COLOR_BGR2RGB)
            display(Image.fromarray(rgb))

    total_time = time.perf_counter() - t_start
    timed_frames = len(times)

    # ─────────────────────────────── Summary ─
    ms = [t * 1_000 for t in times]
    if timed_frames == 0:
        print("\n[WARNING] fewer frames than WARMUP_FRAMES; nothing timed.")
        sys.exit(0)

    print("\n───── Speed-test summary (warm-up skipped) ─────")
    print(f"Total frames           : {len(paths)}")
    print(f"Warm-up frames ignored : {WARMUP_FRAMES}")
    print(f"Frames timed           : {timed_frames}")
    print(f"Total wall-clock time  : {total_time:,.2f} s")
    print(f"Average / frame        : {statistics.mean(ms):,.2f} ms"
          f"  ({1_000/statistics.mean(ms):,.2f} FPS)")
    print(f"Median                 : {statistics.median(ms):,.2f} ms")
    print(f"Fastest                : {min(ms):,.2f} ms")
    print(f"Slowest                : {max(ms):,.2f} ms")
    print("────────────────────────────────────────────────")
